# This notebook carries out the valididation analysis, comparing the temporal and spatial variation in the identified recreational fishing trips with data from MRIP and TPWD 

## Key inputs: 
* **Station_NonStationAnalysis_full.csv** -- includes an indicator of whether the trip stopped at a station
* **rec_indicators_with_V3** - features used for classification
* **DisappearanceIndicators.csv** - indicators of whether a trip is fully tracked

## Key outputs:
* **Validation figures** for temporal and spatial correlations between mobility data trips and those in data from Texas and the MRIP states of Alabama and Louisiana

# 0. Preliminaries

## Load libraries and functions

In [ ]:
!pip install h3
!pip install --upgrade h3

In [ ]:
# # Install modules
!pip install tqdm
!pip install statsmodels

# # Suppress all warnings
# import warnings
# warnings.filterwarnings("ignore")

import pandas as pd
import os
import sys
import time
import csv
from datetime import datetime, timedelta
from datetime import datetime
import pytz  
import warnings
import numpy as np
import math
local_timezone = pytz.timezone('US/Central')

import csv
import datetime
import datetime as dt
import geopandas as gpd
import heapq
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pytz
import requests
import seaborn as sns
import time
import math
import statsmodels
import inspect
import folium
from folium import plugins
import h3

from scipy import interpolate
from shapely import wkt
from shapely.geometry import Polygon
from zipfile import ZipFile
from shapely.geometry import Point  # Import the Point class from shapely.geometry
from datetime import datetime
from tqdm import tqdm

class OperationCancelled(Exception):
    pass

local_timezone = pytz.timezone('US/Central')

In [ ]:
import warnings
warnings.filterwarnings('ignore')

## Set directories to be used

In [ ]:
#################################################################
# Directory for this output
OurTable_V3_directory = '~/RecFishing/Analysis with Our Tables and V3/Data Files'
# Expand the tilde to the user's home directory
OurTable_V3_directory = os.path.expanduser(OurTable_V3_directory)
# Check to make sure the directory exist
DirExist = os.path.exists(OurTable_V3_directory)
print(OurTable_V3_directory, "exists = " ,DirExist)


#################################################################
# Directory for output from the first draw
Batch01_directory = '~/RecFishing/Analysis with Our Tables and V3/Data Files/Batch01'
# Expand the tilde to the user's home directory
Batch01_directory = os.path.expanduser(Batch01_directory)
# Check to make sure the directory exist
DirExist = os.path.exists(Batch01_directory)
print(Batch01_directory, "exists = " ,DirExist)



#################################################################
# Directory for Groups of V3 Pings
V3_Pings_Groups_directory = '~/RecFishing/Analysis with Our Tables and V3/Data Files/V3_Ping_Groups'
V3_Pings_Groups_directory = os.path.expanduser(V3_Pings_Groups_directory)
print(V3_Pings_Groups_directory, "exists = " ,os.path.exists(V3_Pings_Groups_directory))

#################################################################
# Directory some core data fro analysis
CoreData_Directory = '~/RecFishing/CoreData'
CoreData_Directory = os.path.expanduser(CoreData_Directory)
print(CoreData_Directory, "exists = ", os.path.exists(CoreData_Directory))

#################################################################
# Directory  with Original_directory material
Original_directory = '~/RecFishing/DataflowStudioJobs'
Original_directory = os.path.expanduser(Original_directory)
DirExist = os.path.exists(Original_directory)
print(Original_directory, "exists = ", DirExist)


#################################################################
# Directory  with Original_directory material
Previously_Processed_directory = '~/RecFishing/DataflowStudioJobs/FinalCode - Rec Fishing Identification'
Previously_Processed_directory = os.path.expanduser(Previously_Processed_directory)
DirExist = os.path.exists(Previously_Processed_directory)
print(Previously_Processed_directory, "exists = ", DirExist)

#################################################################
# Directory with Travel Cost files
Travel_Cost_directory = '~/RecFishing/Travel Costs with Dedicated Table/CSV Files'
# Expand the tilde to the user's home directory
Travel_Cost_directory = os.path.expanduser(Travel_Cost_directory)
DirExist = os.path.exists(Travel_Cost_directory)
print(Travel_Cost_directory, "exists = ", DirExist)

Travel_Cost_Shapefile_directory = '~/RecFishing/Travel Costs with Dedicated Table/Shapefiles'
# Expand the tilde to the user's home directory
Travel_Cost_Shapefile_directory = os.path.expanduser(Travel_Cost_Shapefile_directory)
DirExist = os.path.exists(Travel_Cost_Shapefile_directory)
print(Travel_Cost_Shapefile_directory, "exists = ", DirExist)


#################################################################
# Directory with Weather data and related files
Weather_Data_directory = '~/RecFishing/uploaded_files/Weather Data'
# Expand the tilde to the user's home directory
Weather_Data_directory = os.path.expanduser(Weather_Data_directory)
print(Weather_Data_directory, "exists = ", DirExist)


#################################################################
# Directory with other Uploaded data 
Uploaded_Data_directory = '~/RecFishing/uploaded_files'
Uploaded_Data_directory = os.path.expanduser(Uploaded_Data_directory)
# FishBrain Data
Fishbrain_directory = '~/RecFishing/uploaded_files/FishBrain'
Fishbrain_directory = os.path.expanduser(Fishbrain_directory)

# FishAngler Data
FishAngler_directory = '~/RecFishing/uploaded_files/FishAngler'
FishAngler_directory = os.path.expanduser(FishAngler_directory)

#################################################################
# Results and Analysis
Results_directory = '~/RecFishing/Analysis with Our Tables and V3/Results'
Results_directory = os.path.expanduser(Results_directory)

Figures_directory = '~/RecFishing/Analysis with Our Tables and V3/Results/Figures'
Figures_directory = os.path.expanduser(Figures_directory)

TrajectoryMaps_directory = '~/RecFishing/Analysis with Our Tables and V3/Results/Figures/Maps'
TrajectoryMaps_directory = os.path.expanduser(TrajectoryMaps_directory)

Validation_directory = '~/RecFishing/Analysis with Our Tables and V3/Results/Validation'
Validation_directory = os.path.expanduser(Validation_directory)


####################################################################################
####################  AIS Directory #################################################
AIS_Directory = '~/RecFishing/AIS Files/Data'
AIS_Directory = os.path.expanduser(AIS_Directory)
DirExist = os.path.exists(AIS_Directory)
print(AIS_Directory, "exists = ", DirExist)

# ID_list_RandomSample from ScheduledExecution5.pkl


## Set input and output files to be used

In [ ]:
def check_file_existence(file_path):
    if not os.path.exists(file_path):
        print(f"{file_path} Does NOT Exist")


######################################################################################################################
#########################  Log File  ################
Log_filename  =  os.path.join(OurTable_V3_directory, 'Log.txt')

######################################################################################################################
########################## Complete list of randomized IDs- without bernouli sampling 740k #########################
# PKL_File_With_Random_IDs_Filename  =  os.path.join(Original_directory, 'cuebiq_id_list_wo_sampling_740k.pkl')
PKL_File_With_Random_IDs_Filename  =  os.path.join(CoreData_Directory, 'cuebiq_id_list_wo_sampling_740k.pkl')
check_file_existence(PKL_File_With_Random_IDs_Filename)
    
# Data gathered and used prior to the NOAA Webinar in February 2024
IDs_Used_in_NOAA_Webinar_filename = os.path.join(CoreData_Directory, 'IDs_From_Random_Draw_Prior_to_NOAA_Webinar.csv')
check_file_existence(IDs_Used_in_NOAA_Webinar_filename)
Ping_Used_in_NOAA_Webinar_filename = os.path.join(CoreData_Directory, 'Pings_From_Random_Draw_Prior_to_NOAA_Webinar.csv')
check_file_existence(Ping_Used_in_NOAA_Webinar_filename)
                                                     
# List of IDs that have been processed for Indicators
AlreadyFullyProcessedIDs_Filename  =  os.path.join(OurTable_V3_directory, 'RandomlyChosenCuebiq_ids.List_of_Processed_ids.csv')
check_file_existence(AlreadyFullyProcessedIDs_Filename)
    
######################################################################################################################
#########################  ID Checklist with columns for ID, Pings, Indicators Created (TF) & Trips  ################
IDs_Pulled_from_Dedicated_Table_filename  =  os.path.join(OurTable_V3_directory, 'IDs_Pulled_From_Dedicated_Table.csv')
check_file_existence(IDs_Pulled_from_Dedicated_Table_filename)
    
ID_For_V3_Queries_filename  =  os.path.join(OurTable_V3_directory, 'IDs_from_V3.csv')
check_file_existence(ID_For_V3_Queries_filename)
    
RecTripRating_filename =  os.path.join(OurTable_V3_directory, 'RecTripRating.csv')
check_file_existence(RecTripRating_filename)
    
# This file contains information about the rows of Pings_V3_temp_filename that can be used to avoid loading the entire file into a data frame
V3_Pings_Index_filename =  os.path.join(OurTable_V3_directory, 'V3_Pings_File_Index.csv')
check_file_existence(V3_Pings_Index_filename)
    
ID_Groups_filename = os.path.join(OurTable_V3_directory,'Cuebiq_ID_Groups.csv')
check_file_existence(ID_Groups_filename)

######################################################
# Pings in the OurTable for a single large draw of IDs TEMPORARY FILE
Pings_OurTable_temp_filename = os.path.join(OurTable_V3_directory,'Pings_OurTable_temp.csv')
check_file_existence(Pings_OurTable_temp_filename)
    
# Pings from V3 corresponding with the IDs found in the OurTable 
Pings_V3_temp_filename = os.path.join(OurTable_V3_directory,'Pings_V3_temp.csv')
check_file_existence(Pings_V3_temp_filename)
    
# Set output file names
Indicators_IDs_checked_filename = os.path.join(OurTable_V3_directory, 'IDs_Checked_Indicators_OurTable.csv')
check_file_existence(Indicators_IDs_checked_filename)

cuebiq_id_list_and_count_filename= os.path.join(OurTable_V3_directory,'cuebiq_id_list_and_count.csv')
check_file_existence(cuebiq_id_list_and_count_filename)

# List of IDs and dates for V3 query Pings in the OurTable for a single large draw of IDs TEMPORARY FILE
OurTable_IDs_and_Dates_filename = os.path.join(OurTable_V3_directory,'OurTable_IDs_and_Dates.csv')
check_file_existence(OurTable_IDs_and_Dates_filename)
    
##########################################################################################
############################### PINGS FILES   ##############################################
Pings_OurTable_Gulf_filename= os.path.join(OurTable_V3_directory,'Pings_OurTable_Gulf_ALL.csv')
check_file_existence(Pings_OurTable_Gulf_filename)

Combined_Pings_OurTable_Gulf_filename= os.path.join(OurTable_V3_directory,'Combined_Pings_OurTable_Gulf_ALL.csv')
check_file_existence(Combined_Pings_OurTable_Gulf_filename)

Pings_V3_Before_After_filename= os.path.join(OurTable_V3_directory,'Pings_V3_Before_After.csv')
check_file_existence(Pings_V3_Before_After_filename)

Combined_Pings_V3_Before_After_filename= os.path.join(OurTable_V3_directory,'Combined_Pings_V3_Before_After.csv')
check_file_existence(Combined_Pings_V3_Before_After_filename)

Pings_OurTable_Gulf_MT19_filename= os.path.join(OurTable_V3_directory,'Pings_OurTable_Gulf_MT19.csv')
check_file_existence(Pings_OurTable_Gulf_MT19_filename)

Pings_OurTable_Coast_filename= os.path.join(OurTable_V3_directory,'Pings_OurTable_Coast.csv')
check_file_existence(Pings_OurTable_Coast_filename)

Pings_OurTable_Outside_our_wkts_filename= os.path.join(OurTable_V3_directory,'Pings_OurTable_Outside_our_wkts.csv')
check_file_existence(Pings_OurTable_Outside_our_wkts_filename)
    
##########################################################################################
############################### INDICATORS  ##############################################
Indicators_filename = os.path.join(OurTable_V3_directory,'Indicators_OurTable.csv')
check_file_existence(Indicators_filename)
    
# cuebiq_id_count_filename= os.path.join(EEZ_V3_directory,'cuebiq_id_count_distribution_EEZ_V3.csv')
Indicators_Classified_filename = os.path.join(OurTable_V3_directory,'Indicators_OurTable.Predictions.csv')
check_file_existence(Indicators_Classified_filename)

Rec_Indicators_filename = os.path.join(OurTable_V3_directory,'Rec_Indicators_OurTable.csv')
check_file_existence(Rec_Indicators_filename)

Rec_Indicators_Step1_filename = os.path.join(OurTable_V3_directory,'Rec_Indicators_OurTable.Step1.csv')
check_file_existence(Rec_Indicators_filename)

V3_Indicators_filename =  os.path.join(OurTable_V3_directory,'V3_indicators.csv')
check_file_existence(V3_Indicators_filename)

Rec_indicators_with_V3_filename = os.path.join(OurTable_V3_directory,'rec_indicators_with_V3.csv')
check_file_existence(Rec_indicators_with_V3_filename)

Indicators_with_V3_indicators_filename= os.path.join(OurTable_V3_directory,'Indicators_with_V3_indicators_indicators.csv')
check_file_existence(Indicators_with_V3_indicators_filename)

# Rec_Indicators_Selected_filename = os.path.join(OurTable_V3_directory,'Rec_Indicators_OurTable_Selected.csv')
Rec_Indicators_Selected_filename = os.path.join(OurTable_V3_directory,'Rec_Indicators_OurTable_Selected_May2024.csv')
check_file_existence(Rec_Indicators_Selected_filename)

Rec_Indicators_Final_All_Exclusions_And_Disappearance_filename = os.path.join(OurTable_V3_directory,'Rec_Indicators_Final_All_Exclusions_And_Disappearance.csv')
check_file_existence(Rec_Indicators_Final_All_Exclusions_And_Disappearance_filename)

# Sorted_Results_file_path = os.path.join(OurTable_V3_directory,'Indicators_EEZ_and_V3.Predictions.sorted.csv')
# RecFishing_Results_file_path =  os.path.join(OurTable_V3_directory,'RecFishingBoat Predictions.sorted.csv')

DisappearanceIndicators_filename = os.path.join(OurTable_V3_directory,'DisappearanceIndicators.csv')
check_file_existence(DisappearanceIndicators_filename)

DisappearanceAnalysis_filename = os.path.join(OurTable_V3_directory,'DisappearanceAnalysis.csv')
check_file_existence(DisappearanceAnalysis_filename)

Stops_Indicators_filename = os.path.join(OurTable_V3_directory,'Stops_Indicators.csv')
check_file_existence(Stops_Indicators_filename)

Trawls_Indicators_filename = os.path.join(OurTable_V3_directory,'Trawls_Indicators.csv')
check_file_existence(Trawls_Indicators_filename)

Stop_Trawls_Indicators_filename = os.path.join(OurTable_V3_directory,'Stop_Trawls_Indicators.csv')
check_file_existence(Stop_Trawls_Indicators_filename)

Combined_Stops_Indicators_filename = os.path.join(OurTable_V3_directory,'Combined_Stops_Indicators.csv')
check_file_existence(Combined_Stops_Indicators_filename)

Combined_Trawls_Indicators_filename = os.path.join(OurTable_V3_directory,'Combined_Trawls_Indicators.csv')
check_file_existence(Combined_Trawls_Indicators_filename)

Combined_Stop_Trawls_Indicators_filename = os.path.join(OurTable_V3_directory,'Combined_Stop_Trawls_Indicators.csv')
check_file_existence(Combined_Stop_Trawls_Indicators_filename)

##########################################################################################
############################### Files that COMBINE BATCH01 and newer data ################
Combined_indicators_with_disappearance_filename = os.path.join(OurTable_V3_directory,'Combined_indicators_with_disappearance.csv')
check_file_existence(Combined_indicators_with_disappearance_filename)

#################################################################
####################   Results and Analysis #################################################
Station_NonStationAnalysis_filename  = os.path.join(Results_directory,'Station_NonStationAnalysis.csv')
check_file_existence(Station_NonStationAnalysis_filename)

Station_NonStationAnalysis_full_filename  = os.path.join(Results_directory,'Station_NonStationAnalysis_full.csv')
check_file_existence(Station_NonStationAnalysis_full_filename)


#################################################################
####################   Validation Files #################################################
ValidationAnalysisCompleteRecTrips_filename = os.path.join(Validation_directory, 'ValidationAnalysisCompleteRecTrips.csv')
check_file_existence(Station_NonStationAnalysis_full_filename)

ValidationAnalysisNonRecTrips_filename = os.path.join(Validation_directory, 'ValidationAnalysisComplete_Non371_Trips.csv')
check_file_existence(ValidationAnalysisNonRecTrips_filename)

Non_Rec_Trips_checked_filename = os.path.join(Validation_directory, 'Non_Rec_Trips_checked.csv')
check_file_existence(Non_Rec_Trips_checked_filename)

ValidationAnalysis_sites_With_Stops_filename = os.path.join(Validation_directory,'ValidationAnalysis_sites_With_Stops.csv')
check_file_existence(Non_Rec_Trips_checked_filename)

##########################################################################################
############################### WEATHER data files ####################################
Buoys_file_path  = os.path.join(Weather_Data_directory,'Buoys.csv')
check_file_existence(Buoys_file_path)

Weather_file_path  = os.path.join(Weather_Data_directory,'DailyWeatherData.csv')
check_file_existence(Weather_file_path)

##########################################################################################
############################### SUPPLEMENTARY MAP DATA  ############################
Industrial_polygons_filename  = os.path.join(Uploaded_Data_directory,'Polygons Around Industrial Sites.wkt')
check_file_existence(Industrial_polygons_filename)


station_points_filename = os.path.join(Uploaded_Data_directory,'LA_TX_Union_Station_WGS84.csv')
check_file_existence(station_points_filename)

MRIP_station_points_filename =os.path.join(Uploaded_Data_directory,'MRIP_stations.csv')
check_file_existence(MRIP_station_points_filename)

FishBrain_filename =os.path.join(Fishbrain_directory,'Fishbrain_public_offshore_catches_at_gulfofmexico_2019-2021_shared-dataset.csv')
check_file_existence(FishBrain_filename)

FishAngler_filename =os.path.join(FishAngler_directory,'captures-2018-to-2022.csv')
check_file_existence(FishAngler_filename)

# TX_Roving_Station_Lists_filename = os.path.join(Uploaded_Data_directory,'Fishing Site List - Roving and Creel Sites (from Stata) - with state, county, and zcta codes - final (from Jesse Backstrom).csv')
# check_file_existence(TX_Roving_Station_Lists_filename)
TX_Roving_Counts_filename = os.path.join(Uploaded_Data_directory,'TPWD roving counts.csv')
check_file_existence(TX_Roving_Counts_filename)

MRIP_trips_filename = os.path.join(Uploaded_Data_directory,'AL_MS_MRIPTripData.csv')
check_file_existence(MRIP_trips_filename)


Gardner_Point_filename = os.path.join(Travel_Cost_Shapefile_directory,'GOM_AR_points.csv')
check_file_existence(TX_Roving_Counts_filename)

##########################################################################################
############################### AIS Files INCLUDING CLASSIFIER ############################
RF_Classfier_filename = os.path.join(AIS_Directory, 'rf_model_AIS_2019.pkl')
check_file_existence(RF_Classfier_filename)

RF_Importance_Factors_filename = os.path.join(AIS_Directory, 'rf_classifier_importance_factors.csv')
check_file_existence(RF_Importance_Factors_filename)

AIS_Predictions_filename = os.path.join(AIS_Directory, 'RandomForest_Predictions2019AISData.csv')
check_file_existence(AIS_Predictions_filename)


### Dedicate Table Names for reference
# Dedicated table with all Pings within the Gulf WKT for 1/12019 - 4/22/2022
#  Table Name:  dedicated.ScheduledExecution5.DeviceTable   
#  Code used for call:  RecFishing/DataflowStudioJobs/ScheduledEx5-updated.ipynb

# Dedicated table with all Pings within the Gulf WKT AND Origin for 1/12019 - 4/22/2022
#  Table Name:  dedicated.ScheduledExecution5_parallel_origin.DeviceTable
#  Code used for call:  RecFishing/DataflowStudioJobs/ScheduledEx5-origin.ipynb

##########################################################################################
############################### Results & Analysis ############################


## Date and Distance Functions 

In [ ]:
# THIS CODE GENERATE THE SEASON BASED ON MONTH

# def month_to_season(month_num):
    
#     monthMar= (month_num>1)*(month_num-1) + (month_num<=1)*(11+month_num)
#     season=int((monthMar) / 3)+1
    
#     return season
# month 6 and 1 are wrong

def month_to_season(month_num):
    if (month_num == 11) | (month_num == 0) | (month_num == 1): # DEC-FEB
        season = 4
    if (month_num == 2) | (month_num == 3) | (month_num == 4): # MARCH - MAY
        season = 1
    if (month_num == 5) | (month_num == 6) | (month_num == 7): # JUNE - August 
        season = 2
    if (month_num == 8) | (month_num == 9) | (month_num == 10): # SEP- NOV
        season = 3
    return season

# check season function
season= month_to_season(1)
season

# note 0 equals Jan and 11 equals december 
def monthofyear(EpochTime):  # January = 0
    Base = 1546322400  # Jan. 1, 2019, 12:00 a.m. 
    Jan12021 = 731
    Jan12022 = 1096
    Jan12023 = 1461

    FebStart = 31
    MarStart = 59
    AprStart = 90
    MayStart = 120
    JunStart = 151
    JulStart = 181
    AugStart = 212
    SepStart = 243
    OctStart = 273
    NovStart = 304
    DecStart = 334
    leapyear2020 = 1582869600     # Feb 29, 2020

    days_since_2019 = epoch_to_days_since_1_1_2019(EpochTime)
    leapyearadjust = -1*(EpochTime >leapyear2020)
    year = int((days_since_2019+ leapyearadjust)/365)
    dayofyear = (days_since_2019) - 365*year +  leapyearadjust
    month = 1
    month = 1*(dayofyear>=FebStart) + \
            1*(dayofyear>=    MarStart ) + \
            1*(dayofyear>=    AprStart ) + \
            1*(dayofyear>=    MayStart ) + \
            1*(dayofyear>=    JunStart ) + \
            1*(dayofyear>=    JulStart ) + \
            1*(dayofyear>=    AugStart ) + \
            1*(dayofyear>=    SepStart ) + \
            1*(dayofyear>=    OctStart ) + \
            1*(dayofyear>=    NovStart ) + \
            1*(dayofyear>=    DecStart )
    return month
        

def epoch_to_season(EpochTime):
    month = monthofyear(EpochTime)
    season = month_to_season(month)
    return season

def epoch_to_DOW(EpochTime):
    Base = 1546322400  # Jan. 1, 2019, 12:00 a.m. 
    BaseDOW = 1
    DaySince = (EpochTime - Base) / (24*60*60)
    WeeksSinceBase = DaySince / 7
    DayOfWeek = BaseDOW + int((WeeksSinceBase - int(WeeksSinceBase)) * 7)
    return DayOfWeek

def epoch_to_days_since_1_1_2019(EpochTime):
    Base = 1546322400  # Jan. 1, 2019, 12:00 a.m. 
    DaySinceBase = int((EpochTime-Base) / (60*60*24))
    return DaySinceBase

def AISdate_to_epoch(AISDate):
    from datetime import datetime

    # date_string = "2019-06-01T16:14:14"

    # Define the format of the input date string
    date_format = "%Y-%m-%dT%H:%M:%S"

    # Convert the date string to a datetime object
    dt_object = datetime.strptime(AISDate, date_format)

    # Convert the datetime object to epoch time
    epoch_time = int(dt_object.timestamp())

    # print("Epoch Time:", epoch_time)
    return epoch_time

def epoch_to_hour_of_day(EpochTime):
    Base = 1546322400  # Jan. 1, 2019, 12:00 a.m. 
    # Daylight Savings Time points for US Central Time
    start2019 = 1552204800  
    end2019 = 1572768000
    start2020 = 1583654400
    end2020 = 1604217600
    start2021 = 1615708800
    end2021 = 1636272000
    start2022 = 1647158400
    end2022 = 1667721600
    start2023 = 1678608000
    end2023 = 1699171200

    
    DayLightSavingsAdjust = +1 * (EpochTime > start2019) + \
                            -1 * (EpochTime > end2019) + \
                            +1 * (EpochTime > start2020) + \
                            -1 * (EpochTime > end2020) + \
                            +1 * (EpochTime > start2021) + \
                            -1 * (EpochTime > end2021) + \
                            +1 * (EpochTime > start2022) + \
                            -1 * (EpochTime > end2022) + \
                            +1 * (EpochTime > start2023) + \
                            -1 * (EpochTime > end2023)
#    print(DayLightSavingsAdjust)
    DaysSince = ((EpochTime-Base) / (60*60*24))
    PortionOfDay = DaysSince - int(DaysSince)
    HourOfDay = int(PortionOfDay*24) + DayLightSavingsAdjust
    return HourOfDay

def epoch_to_date(epoch_time):
    if isinstance(epoch_time, pd.Series):
        # If input is a Series, apply the function to each element
        return epoch_time.apply(epoch_to_date)
    else:
        # Convert epoch time to a datetime object
        epoch_time = int(epoch_time)
        dt = datetime.fromtimestamp(epoch_time)

        # Format the datetime as 'YYYY-MM-DD'
        formatted_date = dt.strftime('%Y-%m-%d')
        return formatted_date
    
# def epoch_to_date(epoch_time):
#     # Convert epoch time to a datetime object
#     epoch_time=int(epoch_time)
#     dt = datetime.fromtimestamp(epoch_time)
#     print(epoch_time, dt)
    
#     # Format the datetime as 'YYYY-MM-DD'
#     formatted_date = dt.strftime('%Y-%m-%d')
    
#     return formatted_date

def epoch_to_datetime(epoch_time):
    return datetime.fromtimestamp(epoch_time)

def date_to_epoch(date_string, date_format='%m/%d/%Y'):
    """
    Convert a date string to epoch time integer.
    
    Parameters:
        date_string (str): The date string to convert.
        date_format (str): The format of the date string. Default is '%m/%d/%Y'.
    
    Returns:
        int: Epoch time integer.
    """
    # Parse the date string to a datetime object
    date_obj = datetime.strptime(date_string, date_format)
    
    # Convert the datetime object to epoch time
    epoch_time = int(date_obj.timestamp())
    
    return epoch_time

In [ ]:
def get_dates_sequence(
    date_start, 
    date_end, 
    date_format
):
    return [
        (datetime.strptime(date_start, date_format) + timedelta(days=x)).strftime(date_format)
        for x in range (
        0,
        (datetime.strptime(date_end, date_format) - datetime.strptime(date_start, date_format) + timedelta(days=1)).days
        )
    ]

In [ ]:
date_format = "%Y%m%d"

first_date = "20190101"
last_date_to_compute = "20220422"

In [ ]:
def haversine(lat1, lon1, lat2, lon2, to_radians=True, earth_radius=6371):
    if to_radians:
        lat1, lon1, lat2, lon2 = np.radians([lat1, lon1, lat2, lon2])

    a = np.sin((lat2-lat1)/2.0)**2 + \
        np.cos(lat1) * np.cos(lat2) * np.sin((lon2-lon1)/2.0)**2

    return earth_radius * 2 * np.arcsin(np.sqrt(a))

In [ ]:
def calculate_speed(df):
    # Sort the DataFrame based on 'event_timestamp'
    df.sort_values(by='event_timestamp', inplace=True)
    
    # Calculate distances and speeds
    df['dist_fwd'] = haversine(df['lat'], df['lng'], df['lat'].shift(1), df['lng'].shift(1))
    df['time_fwd'] = df['event_timestamp'] - df['event_timestamp'].shift(1)
    df['ping_speed_fwd'] = 60 * abs(df['dist_fwd'] / (0.000001 + df['time_fwd']))
    df['ping_speed_fwd'].iloc[0] = 0.0  # KM/minute


## H3 Functions

#### Default resolution for all H3 analysis

In [ ]:
# Resolution 10 has an average edge length of 75.86 meters, meaning, the distance between two opposite vertices is 151.5 meters
resolution = 10

#### Function to get the h3 cell for a complete data frame

In [ ]:
!pip install h3
from shapely.geometry import Point
import h3

import h3
import pandas as pd

# Optimized function to get H3 cells for a GeoDataFrame and return both the set and modified dataframe
def get_h3_cells_set_and_dataframe(dataframe, resolution):
    # Calculate the H3 cells for all rows in one go
    dataframe['h3_cell'] = [
        h3.latlng_to_cell(lat, lng, resolution) 
        for lat, lng in zip(dataframe['lat'], dataframe['lng'])
    ]
    
    # Create the set of unique H3 cells directly from the 'h3_cell' column
    h3_cells = set(dataframe['h3_cell'])
    
    return h3_cells, dataframe

# Function to get H3 cells for a GeoDataFrame
def get_h3_cells_for_dataframe(dataframe, resolution):
    h3_cells = set()
    for index, row in dataframe.iterrows():
        point = Point(row['lng'], row['lat'])
        h3_cells.add(h3.latlng_to_cell(point.y, point.x, resolution))
    return h3_cells

### Function to add columns with h3_cell and neighbors to a data frame

In [ ]:
# Function to get H3 cells and their neighbors
def add_h3_and_neighbors_to_dataframe(dataframe, resolution):
    # Calculate H3 cells for each row (assuming 'lat' and 'lng' columns are present)
    dataframe['h3_cell'] = [
        h3.latlng_to_cell(lat, lng, resolution) 
        for lat, lng in zip(dataframe['lat'], dataframe['lng'])
    ]
    
    # Function to get the neighbors for each H3 cell
    def get_neighbors(h3_cell):
        return list(h3.grid_disk(h3_cell, 1))  # 1 indicates the 1-ring neighbors

    # Add a column for neighbors
    dataframe['h3_neighbors'] = dataframe['h3_cell'].apply(get_neighbors)
    
    return dataframe

# Example usage with AL_MRIP_station_points_df and a resolution of 9
TEST_df= pd.DataFrame({
    'lat': [37.7749, 34.0522, 40.7128],  # Example latitudes
    'lng': [-122.4194, -118.2437, -74.0060]  # Example longitudes
})

# Add H3 cell and neighbors to the dataframe
TEST_df = add_h3_and_neighbors_to_dataframe(TEST_df, resolution)

# Print the dataframe with the new columns
# print(TEST_df)


### Function that compares columns in two data frames

In [ ]:
def CompareColumsInTwoDataFrames(df1, df2):
    # Get the current frame
    frame = inspect.currentframe()
    # Get the arguments from the caller's frame
    args, _, _, values = inspect.getargvalues(frame.f_back)

    # Extract the names of the arguments
    df1_name = [name for name in values if values[name] is df1][0]
    df2_name = [name for name in values if values[name] is df2][0]

    columns_only_in_1 = list(set(df1.columns) - set(df2.columns))
    columns_only_in_1 = sorted(columns_only_in_1)

    # Get the list of columns in Batch01_merged_df that are not in indicators_df
    columns_only_in_2 = list(set(df2.columns) - set(df1.columns))
    columns_only_in_2 = sorted(columns_only_in_2)

    common_in_both  = list(set(df1.columns).intersection(set(df2.columns)))
    common_in_both = sorted(common_in_both)
    
    if len(columns_only_in_1)>0:
        print(" ")
        print("Columns in ", df1_name, "that aren't in", df2_name,":")
        print(columns_only_in_1)
    else:
        print("There are no columns in ", df1_name, "that aren't in", df2_name)
    print(" ")
        

    if len(columns_only_in_2)>0:
        print(" ")
        print("Columns in ", df2_name, "that aren't in", df1_name,":")
        print(columns_only_in_2)
    else:
        print("There are no columns in ", df2_name, "that aren't in", df1_name)

    print(" ")
    print("Columns in both", df1_name, "and", df2_name, ":", common_in_both)

# CompareColumsInTwoDataFrames(Batch01_ind1_df, indicators_df)
# indicators_df.head(3)

## Errant pings code

In [ ]:
def haversine(lat1, lon1, lat2, lon2, to_radians=True, earth_radius=6371):
    if to_radians:
        lat1, lon1, lat2, lon2 = np.radians([lat1, lon1, lat2, lon2])

    a = np.sin((lat2-lat1)/2.0)**2 + \
        np.cos(lat1) * np.cos(lat2) * np.sin((lon2-lon1)/2.0)**2

    return earth_radius * 2 * np.arcsin(np.sqrt(a))

def EliminateErrantPingsSpeed(pings_df, mph_limit):
    km_per_min_limit = mph_limit*(0.0268224)
    # Calculate speed moving forward e.g., row 0 is time since previous trip

    pings_df.sort_values(by='event_timestamp', inplace=True)
    pings_df = pings_df.drop_duplicates()


    # create (or recreate) the time difference variables
    pings_df['time_diff_minutes_from_previous'] = pings_df["event_timestamp"].diff()/60.0
    pings_df['time_diff_minutes_from_previous'].fillna(value=0, inplace=True)

    pings_df['time_diff_minutes_to_next'] = pings_df["event_timestamp"].diff(-1)/60.0
    pings_df['time_diff_minutes_to_next'].fillna(value=99999, inplace=True)

    
    pings_df_shifted_down = pings_df.shift(1)
    pings_df['dist_fwd'] = haversine(pings_df['lat'], pings_df['lng'], pings_df_shifted_down['lat'], pings_df_shifted_down['lng'])
    pings_df['ping_speed_fwd'] = abs(pings_df['dist_fwd']/(0.00001+pings_df['time_diff_minutes_from_previous']))
    pings_df['ping_speed_fwd'].fillna(value=0, inplace=True)

    # Calculate speed moving backward e.g., first row is the speed to the next ping
    pings_df_shifted_up = pings_df.shift(-1)
    pings_df['dist_bkwd'] = haversine(pings_df['lat'], pings_df['lng'], pings_df_shifted_up['lat'], pings_df_shifted_up['lng'])
    pings_df['ping_speed_bkwd'] = abs(pings_df['dist_bkwd']/(0.00001+pings_df['time_diff_minutes_to_next']))
    pings_df['ping_speed_bkwd'].fillna(value=0, inplace=True)

    pings_df['Avg_ping_speed'] = (pings_df['ping_speed_fwd'] + pings_df['ping_speed_bkwd']) / 2
    pings_df['row_index'] = pings_df.reset_index().index

    # Step 2: Check if the maximum of ping_speed > km_per_min_limit
    iIteration=0
    while len(pings_df) > 2 and pings_df['Avg_ping_speed'].max() > km_per_min_limit:
        iIteration=iIteration+1

        max_index = pings_df['Avg_ping_speed'].idxmax()
    
        # Step 4: Recalculate ping_speed_fwd for the row after the row that was dropped
        if max_index + 1 < len(pings_df) and max_index - 1 >= 0:
            lat_after = pings_df.iloc[max_index+1]['lat']
            lon_after = pings_df.iloc[max_index+1]['lng']
            lat_before = pings_df.iloc[max_index - 1]['lat']
            lon_before = pings_df.iloc[max_index - 1]['lng']
            distance = haversine(lat_before, lon_before, lat_after, lon_after)
            time_diff = pings_df.iloc[max_index+1]['event_timestamp']-pings_df.iloc[max_index-1]['event_timestamp']
            new_speed = distance /time_diff
            
            # Calculate new fwd speed for the row before
            index_before = max_index - 1
            index_after = max_index + 1
            
            # Update the value using .loc[] or .iloc[] with a single call
            pings_df.loc[index_before, 'ping_speed_fwd'] = new_speed
            pings_df.loc[index_after, 'ping_speed_bkwd'] = new_speed

            pings_df = pings_df[pings_df['event_timestamp'].notna() & (pings_df['event_timestamp'] != '')]

        
        ################ Debugging ###############
        pings_df = pings_df.drop(max_index)
        pings_df['Avg_ping_speed'] = (pings_df['ping_speed_fwd'] + pings_df['ping_speed_bkwd']) / 2

        # Reset index (I don't know if this is really necessary)
        pings_df.reset_index(drop=True, inplace=True)
    
    return pings_df

### Function to merge Trip DFs
Merges two data frames based on cuebiq_id and Trip_number. If the list of IDs is not identical, this returns an empty data frame.

In [ ]:
def Merge_Trip_dfs(df1, df2):
    # Check if the lists of values of cuebiq_id are the same in both data frames
    if set(df1['cuebiq_id']) != set(df2['cuebiq_id']):
        print("The two data frames do not have the same values of cuebiq_id")
        return pd.DataFrame()
    
    # Identify common columns, excluding 'cuebiq_id' and 'Trip_number'
    common_cols = [col for col in df1.columns if col in df2.columns and col not in ['cuebiq_id', 'Trip_number']]
    
    # Drop common columns from df2
    df2 = df2.drop(columns=common_cols)
    
    # Merge the data frames on 'cuebiq_id' and 'Trip_number'
    merged_df = pd.merge(df1, df2, on=['cuebiq_id', 'Trip_number'], how='inner')
    
    return merged_df

### Function that gets H3 neighbors, and the neighbors of the neighbors -- a double ring around the original points

In [ ]:
def get_h3_cells_and_neighbors_two_levels(dataframe, resolution):
    h3_cells_and_neighbors = set()
    for index, row in dataframe.iterrows():
        point = Point(row['lng'], row['lat'])
        h3_cell = h3.latlng_to_cell(point.y, point.x, resolution)
        neighbors = h3.grid_disk(h3_cell, 1)
        
        # Add the original H3 cell and its neighbors at the current resolution
        h3_cells_and_neighbors.add(h3_cell)
        h3_cells_and_neighbors.update(neighbors)
        
        # Add neighbors of neighbors at the same resolution
        for neighbor in neighbors:
            second_layer_neighbors = h3.grid_disk(neighbor, 1)
            h3_cells_and_neighbors.update(second_layer_neighbors)

    return h3_cells_and_neighbors

def get_h3_cells_and_neighbors(dataframe, resolution):
    h3_cells_and_neighbors = set()
    for index, row in dataframe.iterrows():
        point = Point(row['lng'], row['lat'])
        h3_cell = h3.latlng_to_cell(point.y, point.x, resolution)
        neighbors = h3.grid_disk(h3_cell, 1)

        # Add both the original H3 cell and its neighbors at lower resolution to the set
        h3_cells_and_neighbors.add(h3_cell)
        h3_cells_and_neighbors.update(neighbors)

    return h3_cells_and_neighbors



## Load Auxiliary Files -- polygons, stations, etc.

### Load stations (MRIP & TPWD) and find H3 cells and neighbors to those cells

In [ ]:
# Load your dataframes

# Texas -- Data provided Mark Fisher <Mark.Fisher@tpwd.texas.gov> on 6/3/2022
TX_station_points_df = pd.read_csv('../../uploaded_files/TPWD Stations.csv', encoding='latin-1')
TX_station_points_df.rename(columns={'y': 'lat', 'x': 'lng'}, inplace=True)
TX_station_points_df.dropna(subset=['lat'], inplace=True)  # Drop empty rows

# MRIP MS, AL & FL -- downloaded from the site NOAA Site Register
MRIP_station_points_df = pd.read_csv('../../uploaded_files/MRIP-sites-LA-AL-MS.csv')
MRIP_station_points_df.rename(columns={'SITE_LAT': 'lat', 'SITE_LONG': 'lng'}, inplace=True)

AL_MRIP_station_points_df = MRIP_station_points_df[MRIP_station_points_df['STATE_CODE'] == 1].copy()
MS_MRIP_station_points_df = MRIP_station_points_df[MRIP_station_points_df['STATE_CODE'] == 28].copy()
AL_MS_MRIP_station_points_df = MRIP_station_points_df[(MRIP_station_points_df['STATE_CODE'] == 28) |
                                                      (MRIP_station_points_df['STATE_CODE'] == 1)].copy()


FL_MRIP_station_points_df = pd.read_csv('../../uploaded_files/FL Sites -- MRIP Site Registry Escambia and Santa Rosa Counties.csv')
FL_MRIP_station_points_df.rename(columns={'SITE_LAT': 'lat', 'SITE_LONG': 'lng'}, inplace=True)
FL_MRIP_station_points_df = FL_MRIP_station_points_df[FL_MRIP_station_points_df['STATUS'] == "Active"].copy()

# LA Creel Stations -- personal communication from Nicole Smith (WLF) <nsmith@wlf.la.gov> on 12/13/23
LA_CREEL_station_points_df = pd.read_csv('../../uploaded_files/LA creel sites-1.csv')
LA_CREEL_station_points_df.rename(columns={'Latitude': 'lat', 'Longitude': 'lng'}, inplace=True)
LA_CREEL_station_points_df = LA_CREEL_station_points_df[LA_CREEL_station_points_df['Active_Flg'] == 1].copy()
LA_CREEL_station_points_df = LA_CREEL_station_points_df[LA_CREEL_station_points_df['lat'] >0].copy()

# All Ports in GOM Gathered from Marine Traffic Website
MarineTrafficPorts_df = pd.read_csv('../../uploaded_files/MarineTrafficPorts.csv')
MarineTrafficPorts_df.rename(columns={'lon': 'lng'}, inplace=True)

LargePorts_Marine_traffic_df = MarineTrafficPorts_df[MarineTrafficPorts_df['MarineTrafficPortType_num'] == 3].copy()
Medium_Anchorage_Marine_traffic_df = MarineTrafficPorts_df[MarineTrafficPorts_df['MarineTrafficPortType_num'] == 4].copy()
Medium_Marina_Marine_traffic_df = MarineTrafficPorts_df[MarineTrafficPorts_df['MarineTrafficPortType_num'] == 5].copy()
Medium_Port_Marine_traffic_df = MarineTrafficPorts_df[MarineTrafficPorts_df['MarineTrafficPortType_num'] == 7].copy()
Small_Marina_Marine_traffic_df = MarineTrafficPorts_df[MarineTrafficPorts_df['MarineTrafficPortType_num'] == 9].copy()
Small_Port_Marine_traffic_df = MarineTrafficPorts_df[MarineTrafficPorts_df['MarineTrafficPortType_num'] == 10].copy()


### Create H3 cells around station points

In [ ]:
# Confirm the resolution before proceedin
print("This cell creates the df's and lists of h3 cells for use elsewhere")
confirmation = input(f"The resolution is set at {resolution}. Are you sure you want to proceed? Type 'Y' to confirm: ").strip().upper()

if confirmation == 'Y':
    print("Creating dataframes and lists")
else:
    print("Operation cancelled.")

# The maps of stations
state_station_points_df = pd.read_csv('../../uploaded_files/LA_TX_Union_Station_WGS84.csv')

# MRIP_station_points_df = pd.read_csv('../uploaded_files/MRIP_stations.csv')
MRIP_station_points_df = pd.read_csv('../../uploaded_files/MRIP-sites-LA-AL-MS.csv')

MRIP_station_points_df.rename(columns={'SITE_LAT': 'lat', 'SITE_LONG': 'lng'}, inplace=True)

# Create sets of H3 cells and their neighbors for each dataframe
state_station_h3_cells_alone = get_h3_cells_for_dataframe(state_station_points_df, resolution)

mrip_station_h3_cells_alone = get_h3_cells_for_dataframe(MRIP_station_points_df, resolution)


state_station_h3_cells = get_h3_cells_and_neighbors(state_station_points_df, resolution)
mrip_station_h3_cells = get_h3_cells_and_neighbors(MRIP_station_points_df, resolution)

TX_station_points_h3_cells = get_h3_cells_and_neighbors(TX_station_points_df, resolution)
AL_MRIP_station_points_h3_cells= get_h3_cells_and_neighbors(AL_MRIP_station_points_df, resolution)
MS_MRIP_station_points_h3_cells= get_h3_cells_and_neighbors(MS_MRIP_station_points_df, resolution)
FL_MRIP_station_points_h3_cells = get_h3_cells_and_neighbors(FL_MRIP_station_points_df, resolution)
LA_CREEL_station_points_h3_cells= get_h3_cells_and_neighbors(LA_CREEL_station_points_df, resolution)
LargePorts_Marine_traffic_h3_cells= get_h3_cells_and_neighbors(LargePorts_Marine_traffic_df, resolution)
Medium_Anchorage_Marine_traffic_h3_cells= get_h3_cells_and_neighbors(Medium_Anchorage_Marine_traffic_df, resolution)
Medium_Marina_Marine_traffic_h3_cells= get_h3_cells_and_neighbors(Medium_Marina_Marine_traffic_df, resolution)
Medium_Port_Marine_traffic_h3_cells= get_h3_cells_and_neighbors(Medium_Port_Marine_traffic_df, resolution)
Small_Marina_Marine_traffic_h3_cells= get_h3_cells_and_neighbors(Small_Marina_Marine_traffic_df, resolution)
Small_Port_Marine_traffic_h3_cells= get_h3_cells_and_neighbors(Small_Port_Marine_traffic_df, resolution)

# Combine all the h3 cells into a single set
combined_h3_cells = state_station_h3_cells | mrip_station_h3_cells
combined_h3_cells_alone = state_station_h3_cells_alone | mrip_station_h3_cells_alone

print("Done")

### State polygons for identifying state of starting point

In [ ]:
from shapely.geometry import Point, Polygon
from shapely.wkt import loads

## WKT strings from <script src="https://gist.github.com/JoshuaCarroll/49630cbeeb254a49986e939a26672e9c.js"></script>

# # Test on a point on a rectangular state
# wkt_string = "POLYGON((-109.0448 37.0004,-102.0424 36.9949,-102.0534 41.0006,-109.0489 40.9996,-109.0448 37.0004,-109.0448 37.0004))"
# colorado_polygon = loads(wkt_string)
# min_lat,min_lng,max_lat,max_lng=colorado_polygon.bounds

# avg_lat = .5*(min_lat+max_lat)
# avg_lng =  .5*(min_lng+max_lng)
# avg_point = Point(avg_lat,avg_lng)

# is_within_polygon = avg_point.within(colorado_polygon)
# print(avg_point,"is_within_COLORADO",is_within_polygon)


# Define the WKT string for the TX polygon
TX_wkt_string = "POLYGON((-106.5715 31.8659,-106.5042 31.7504,-106.3092 31.6242,-106.2103 31.4638,-106.0181 31.3912,-105.7874 31.1846,-105.5663 31.0012,-105.4015 30.8456,-105.0032 30.6462,-104.8521 30.3847,-104.7437 30.2591,-104.6915 30.0738,-104.6777 29.9169,-104.5679 29.7644,-104.5280 29.6475,-104.4044 29.5603,-104.2067 29.4719,-104.1559 29.3834,-103.9774 29.2948,-103.9128 29.2804,-103.8208 29.2481,-103.5640 29.1378,-103.4692 29.0682,-103.3154 29.0105,-103.1616 28.9601,-103.0957 29.0177,-103.0298 29.1330,-102.8677 29.2157,-102.8979 29.2565,-102.8375 29.3570,-102.8004 29.4898,-102.7002 29.6881,-102.5134 29.7691,-102.3843 29.7596,-102.3047 29.8788,-102.1509 29.7834,-101.7004 29.7572,-101.4917 29.7644,-101.2939 29.6308,-101.2582 29.5269,-101.0056 29.3642,-100.9204 29.3056,-100.7707 29.1642,-100.7007 29.0946,-100.6306 28.9012,-100.4974 28.6593,-100.3601 28.4675,-100.2969 28.2778,-100.1733 28.1882,-100.0195 28.0526,-99.9344 27.9435,-99.8438 27.7638,-99.7119 27.6641,-99.4812 27.4839,-99.5375 27.3059,-99.4290 27.1948,-99.4455 27.0175,-99.3164 26.8829,-99.2065 26.6867,-99.0967 26.4116,-98.8138 26.3574,-98.6668 26.2257,-98.5474 26.2343,-98.3276 26.1357,-98.1697 26.0457,-97.9143 26.0518,-97.6643 26.0050,-97.4020 25.8419,-97.3526 25.9074,-97.0148 25.9679,-97.0697 26.1789,-97.2249 26.8253,-97.0752 27.4230,-96.6096 28.0599,-95.9285 28.4228,-95.3036 28.7568,-94.7296 29.0742,-94.3355 29.3810,-93.8205 29.6021,-93.9317 29.8013,-93.8136 29.9157,-93.7230 30.0489,-93.6996 30.1214,-93.7216 30.2021,-93.7038 30.2792,-93.7628 30.3278,-93.7587 30.3835,-93.7010 30.4380,-93.7024 30.5079,-93.7299 30.5362,-93.6694 30.6296,-93.6090 30.7466,-93.5527 30.8114,-93.5747 30.8834,-93.5307 30.9376,-93.5074 31.0318,-93.5266 31.0812,-93.5335 31.1787,-93.5980 31.1670,-93.6832 31.3055,-93.6708 31.3830,-93.6887 31.4369,-93.7202 31.5107,-93.8315 31.5820,-93.8123 31.6440,-93.8232 31.7188,-93.8342 31.7936,-93.8782 31.8309,-93.9221 31.8869,-93.9661 31.9335,-94.0430 32.0081,-94.0430 33.4681,-94.0430 33.5414,-94.1528 33.5689,-94.1968 33.5872,-94.2627 33.5872,-94.3176 33.5689,-94.3945 33.5597,-94.4275 33.5780,-94.4275 33.6055,-94.4495 33.6421,-94.4879 33.6329,-94.5236 33.6421,-94.6637 33.6695,-94.7461 33.7061,-94.8999 33.7791,-95.0757 33.8818,-95.1526 33.9251,-95.2254 33.9604,-95.2858 33.8750,-95.5399 33.8841,-95.7568 33.8887,-95.8420 33.8408,-96.0274 33.8556,-96.3528 33.6901,-96.6179 33.8442,-96.5836 33.8898,-96.6673 33.8955,-96.7538 33.8179,-96.8335 33.8613,-96.8774 33.8613,-96.9159 33.9388,-97.0917 33.7392,-97.1645 33.7449,-97.2180 33.8978,-97.3746 33.8225,-97.4611 33.8305,-97.4460 33.8761,-97.6945 33.9798,-97.8648 33.8476,-97.9651 33.8978,-98.0983 34.0299,-98.1752 34.1141,-98.3743 34.1425,-98.4773 34.0640,-98.5529 34.1209,-98.7520 34.1232,-98.9539 34.2095,-99.0637 34.2073,-99.1832 34.2141,-99.2505 34.3593,-99.3823 34.4613,-99.4318 34.3774,-99.5718 34.4160,-99.6158 34.3706,-99.8094 34.4726,-99.9934 34.5631,-100.0017 36.4975,-103.0408 36.5008,-103.0655 32.0011,-106.6168 32.0023,-106.5715 31.8659))"
TX_polygon = loads(TX_wkt_string)

# Define the WKT string for the LA polygon
LA_wkt_string = "POLYGON((-94.0430 33.0225,-93.0048 33.0179,-91.1646 33.0087,-91.2209 32.9269,-91.1220 32.8773,-91.1481 32.8358,-91.1412 32.7642,-91.1536 32.6382,-91.1069 32.5804,-91.0080 32.6093,-91.0904 32.4588,-91.0355 32.4379,-91.0286 32.3742,-90.9064 32.3150,-90.9723 32.2616,-91.0464 32.1942,-91.0739 32.1198,-91.0464 32.0593,-91.1014 31.9918,-91.1865 31.9498,-91.3101 31.8262,-91.3527 31.7947,-91.3925 31.6230,-91.5134 31.6218,-91.4310 31.5668,-91.5161 31.5130,-91.5244 31.3701,-91.5477 31.2598,-91.6425 31.2692,-91.6603 31.2328,-91.5848 31.1917,-91.6287 31.1047,-91.5614 31.0318,-91.6397 30.9988,-89.7336 31.0012,-89.8517 30.6686,-89.7858 30.5386,-89.6347 30.3148,-89.5688 30.1807,-89.4960 30.1582,-89.1843 30.2140,-89.0373 30.1463,-88.8354 30.0905,-88.7421 29.8383,-88.8712 29.5758,-88.9371 29.1833,-89.0359 28.9649,-89.2282 28.8832,-89.4754 28.9048,-89.7418 29.1210,-90.1126 28.9529,-90.6619 28.9120,-91.0355 28.9553,-91.3211 29.1210,-91.9061 29.2864,-92.7452 29.4360,-93.8177 29.6009,-93.8631 29.6749,-93.8933 29.7370,-93.9304 29.7930,-93.9276 29.8216,-93.8370 29.8883,-93.7985 29.9811,-93.7601 30.0144,-93.7106 30.0691,-93.7354 30.0929,-93.6996 30.1166,-93.7271 30.1997,-93.7106 30.2899,-93.7656 30.3350,-93.7601 30.3871,-93.6914 30.4416,-93.7106 30.5102,-93.7463 30.5433,-93.7106 30.5954,-93.6914 30.5906,-93.6859 30.6545,-93.6365 30.6781,-93.6200 30.7513,-93.5925 30.7890,-93.5513 30.8150,-93.5623 30.8645,-93.5788 30.8881,-93.5541 30.9187,-93.5294 30.9423,-93.5760 31.0082,-93.5101 31.0318,-93.5596 31.0906,-93.5321 31.1211,-93.5349 31.1799,-93.5953 31.1658,-93.6282 31.2292,-93.6118 31.2668,-93.6859 31.3044,-93.6694 31.3888,-93.7051 31.4240,-93.6859 31.4427,-93.7573 31.4755,-93.7189 31.5083,-93.8040 31.5411,-93.8425 31.6113,-93.8205 31.6581,-93.7985 31.7071,-93.8480 31.8029,-93.9029 31.8892,-93.9606 31.9149,-94.0430 32.0081,-94.0430 32.7041,-94.0430 33.0225,-94.0430 33.0225))"
LA_polygon = loads(LA_wkt_string)

# Define the WKT string for the MS polygon
MS_wkt_string = "POLYGON((-90.3049 35.0041,-88.1955 35.0075,-88.0994 34.8882,-88.1241 34.7044,-88.2573 33.6661,-88.4756 31.8939,-88.4180 30.8657,-88.3850 30.1594,-88.8327 30.0905,-89.1870 30.2104,-89.4919 30.1570,-89.5757 30.1796,-89.6457 30.3326,-89.7748 30.5232,-89.8531 30.6663,-89.7377 30.9988,-91.6287 30.9988,-91.5601 31.0341,-91.6273 31.1106,-91.5916 31.1658,-91.6589 31.2304,-91.6452 31.2656,-91.5436 31.2609,-91.5271 31.3724,-91.5161 31.4099,-91.5120 31.5071,-91.4502 31.5692,-91.5147 31.6230,-91.3966 31.6253,-91.3513 31.7936,-91.2744 31.8589,-91.1673 31.9755,-91.0767 32.0267,-91.0767 32.1198,-91.0437 32.1942,-91.0107 32.2221,-90.9132 32.3150,-91.0313 32.3742,-91.0217 32.4263,-91.0986 32.4634,-91.0080 32.6070,-91.1096 32.5746,-91.1536 32.6394,-91.1426 32.7226,-91.1426 32.7873,-91.1536 32.8519,-91.1206 32.8796,-91.2195 32.9257,-91.2085 32.9995,-91.2016 33.0444,-91.2016 33.1192,-91.1041 33.1835,-91.1536 33.3397,-91.1646 33.4223,-91.2291 33.4337,-91.2524 33.5414,-91.1838 33.6135,-91.2524 33.6878,-91.1261 33.6969,-91.1426 33.7883,-91.0437 33.7700,-91.0327 33.8339,-91.0657 33.8795,-91.0876 33.9434,-90.9998 33.9889,-90.9229 34.0253,-90.9009 34.0891,-90.9668 34.1345,-90.9119 34.1709,-90.8501 34.1345,-90.9338 34.2277,-90.8267 34.2833,-90.6921 34.3434,-90.6509 34.3774,-90.6152 34.3978,-90.5589 34.4432,-90.5740 34.5179,-90.5823 34.5880,-90.5356 34.7506,-90.5136 34.7913,-90.4532 34.8780,-90.3543 34.8476,-90.2911 34.8702,-90.3062 35.0041,-90.3049 35.0041))"
MS_polygon = loads(MS_wkt_string)

# Define the WKT string for the AL polygon
AL_wkt_string = "POLYGON((-88.1955 35.0041,-85.6068 34.9918,-85.1756 32.8404,-84.8927 32.2593,-85.0342 32.1535,-85.1358 31.7947,-85.0438 31.5200,-85.0836 31.3384,-85.1070 31.2093,-84.9944 31.0023,-87.6009 30.9953,-87.5926 30.9423,-87.6256 30.8539,-87.4072 30.6745,-87.3688 30.4404,-87.5240 30.1463,-88.3864 30.1546,-88.4743 31.8939,-88.1021 34.8938,-88.1721 34.9479,-88.1461 34.9107,-88.1955 35.0041))"
AL_polygon = loads(AL_wkt_string)

# Define the WKT string for the AL polygon
FL_wkt_string = "POLYGON((-87.6050 30.9988,-86.5613 30.9964,-85.5313 31.0035,-85.1193 31.0012,-85.0012 31.0023,-84.9847 30.9364,-84.9367 30.8845,-84.9271 30.8409,-84.9257 30.7902,-84.9147 30.7489,-84.8611 30.6993,-84.4272 30.6911,-83.5991 30.6509,-82.5595 30.5895,-82.2134 30.5682,-82.2134 30.5315,-82.1997 30.3883,-82.1544 30.3598,-82.0638 30.3598,-82.0226 30.4877,-82.0473 30.6308,-82.0514 30.6757,-82.0377 30.7111,-82.0514 30.7371,-82.0102 30.7678,-82.0322 30.7914,-81.9717 30.7997,-81.9608 30.8244,-81.8893 30.8056,-81.8372 30.7914,-81.7960 30.7796,-81.6696 30.7536,-81.6051 30.7289,-81.5666 30.7324,-81.5295 30.7229,-81.4856 30.7253,-81.4609 30.7111,-81.4169 30.7088,-81.2274 30.7064,-81.2357 30.4345,-81.1725 30.3160,-81.0379 29.7763,-80.5861 28.8603,-80.3650 28.4771,-80.3815 28.1882,-79.9255 27.1789,-79.8198 26.8425,-79.9118 26.1394,-79.9997 25.5115,-80.3815 24.8802,-80.8704 24.5384,-81.9250 24.3959,-82.2066 24.4496,-82.3137 24.5484,-82.1997 24.6982,-81.3977 25.2112,-81.4622 25.6019,-81.9456 25.9235,-82.2876 26.3439,-82.5307 26.9098,-82.8342 27.3315,-83.0182 27.7565,-83.0017 28.0574,-82.8548 28.6098,-83.0264 28.9697,-83.2050 29.0478,-83.5318 29.4157,-83.9767 29.9133,-84.1072 29.8930,-84.4409 29.6940,-85.0465 29.4551,-85.3610 29.4946,-85.5807 29.7262,-86.1946 30.1594,-86.8510 30.2175,-87.5171 30.1499,-87.4429 30.3006,-87.3750 30.4256,-87.3743 30.4830,-87.3907 30.5658,-87.4004 30.6344,-87.4141 30.6763,-87.5253 30.7702,-87.6256 30.8527,-87.5912 30.9470,-87.5912 30.9682,-87.6050 30.9964,-87.6050 30.9988))"
FL_polygon = loads(FL_wkt_string)

# Find the min and max bounds for each state
# TX_min_lat,TX_min_lng,TX_max_lat,TX_max_lng=TX_polygon.bounds
# LA_min_lat,LA_min_lng,LA_max_lat,LA_max_lng=LA_polygon.bounds
# MS_min_lat,MS_min_lng,MS_max_lat,MS_max_lng=MS_polygon.bounds
# AL_min_lat,AL_min_lng,AL_max_lat,AL_max_lng=AL_polygon.bounds
# FL_min_lat,FL_min_lng,AL_max_lat,FL_max_lng=FL_polygon.bounds

TX_min_lng,TX_min_lat,TX_max_lng,TX_max_lat=TX_polygon.bounds
LA_min_lng,LA_min_lat,LA_max_lng,LA_max_lat=LA_polygon.bounds
MS_min_lng,MS_min_lat,MS_max_lng,MS_max_lat=MS_polygon.bounds
AL_min_lng,AL_min_lat,AL_max_lng,AL_max_lat=AL_polygon.bounds
FL_min_lng,FL_min_lat,FL_max_lng,FL_max_lat=FL_polygon.bounds


avg_lat = .5*(TX_min_lat+TX_max_lat)
avg_lng = .5*(TX_min_lng+TX_max_lng)
avg_point = Point(avg_lng,avg_lat)

TX_within_polygon = avg_point.within(TX_polygon)
LA_within_polygon = avg_point.within(LA_polygon)
MS_within_polygon = avg_point.within(MS_polygon)
AL_within_polygon = avg_point.within(AL_polygon)
FL_within_polygon = avg_point.within(FL_polygon)

# # Initialize variables
TX = LA = MS = AL = FL = 0
if TX_within_polygon:
    TX = 1
elif LA_within_polygon:
    LA = 1
elif MS_within_polygon:
    MS = 1
elif AL_within_polygon:
    AL = 1
elif FL_within_polygon:
    FL = 1
    
print(avg_point,"is_within_TX",TX_within_polygon)
print(avg_point,"is_within_LA",LA_within_polygon)
print(avg_point,"is_within_MS",MS_within_polygon)
print(avg_point,"is_within_AL",AL_within_polygon)
print(avg_point,"is_within_FL",FL_within_polygon)

print(TX,LA,MS,AL,FL)
print(avg_lat,avg_lng)

# 1. Prepare for analysis

## Load csv files from into data frames

In [ ]:
# # Input files
# # Input files
disappear_df = pd.read_csv(DisappearanceIndicators_filename)
indicators_df = pd.read_csv(Rec_indicators_with_V3_filename)

rec_indicators_df = pd.merge(
    disappear_df,
    indicators_df,
    on=['cuebiq_id', 'Trip_number'],
    how='outer',  # Use 'outer' to keep all rows from both DataFrames
)

print(len(rec_indicators_df))

### Load ping files -- commented out unless absolutely needed

In [ ]:
# # Ping Files
# # Origins data created above
# V3_Pings_df = pd.read_csv(Combined_Pings_V3_Before_After_filename)
# Pings_OurTable_Gulf_df= pd.read_csv(Combined_Pings_OurTable_Gulf_filename)

#### Load the indicators for all of the trips from the original 300K cuebiq_id's

In [ ]:
All_Indicators_df = pd.read_csv(Indicators_Classified_filename)
print("There were ", len(All_Indicators_df), "trips for which indicators were developed")
Non_371_indicators_df = All_Indicators_df[All_Indicators_df['Predicted_Class'] != 371]
print("Of these, ", len(Non_371_indicators_df), "trips not identified by ML as a pleasure boat trip (371)")
km_dist_threshhold = 1
Non_371_indicators_df=Non_371_indicators_df[Non_371_indicators_df['Max_distance_traveled_origin_t']>km_dist_threshhold]
print("Of these, ", len(Non_371_indicators_df), "trips went at least ", km_dist_threshhold, " km from the origin and are included for validation analysis.")


# Compare identified recreational trips with the site pressure data

In [ ]:
# # Stations_with_stops_filename
# V3_Pings_df = pd.read_csv(Combined_Pings_V3_Before_After_filename)
# rec_indicators_df = pd.read_csv(Combined_indicators_with_disappearance_filename)

## Redo the stops analysis recovering the stations where stops occurred
The station that is closest in time to the Gulf trip is identified as the station of interest

In [ ]:
%%time
ii = 0
# Load the indicators file that shows whether a trip was stopped near a station
stops_indicators_df = pd.read_csv(Station_NonStationAnalysis_filename)

# Limit the data frame to trips with stops near a station in Alabama or Mississippi
stops_indicators_df = stops_indicators_df[
    (stops_indicators_df['AL_MRIP_station_points_0_1']==1) |
    (stops_indicators_df['MS_MRIP_station_points_0_1']==1) |
    (stops_indicators_df['LA_CREEL_station_points_0_1']==1) |
    (stops_indicators_df['TX_station_points_0_1']==1) 
    ]

# Now take the main rec indicators and pull out only those with identified stops in MS & AL
filtered_rec_indicators_df = rec_indicators_df.merge(
    stops_indicators_df[['cuebiq_id', 'Trip_number']],
    on=['cuebiq_id', 'Trip_number'],
    how='inner'
)

# Drop rows from filtered_rec_indicators_df where cuebiq_id and Trip_number match

print("len(filtered_rec_indicators_df)", len(filtered_rec_indicators_df))

Stations_with_stops_filename = ValidationAnalysis_sites_With_Stops_filename
if os.path.isfile(Stations_with_stops_filename):
    Stations_with_stops_df = pd.read_csv(Stations_with_stops_filename)
    Stations_with_stops_df['cuebiq_id'] = Stations_with_stops_df['cuebiq_id'].astype(int)
    filtered_rec_indicators_df = filtered_rec_indicators_df[
        ~filtered_rec_indicators_df[['cuebiq_id', 'Trip_number']].apply(tuple, axis=1).isin(
            Stations_with_stops_df[['cuebiq_id', 'Trip_number']].apply(tuple, axis=1)
        )
]

Eight_hours = 8 * 60 * 60
One_mph = 0.0268224   # Speeds are calculated in km/min. 1 mph = 0.0268224 km/min

#############################################################################################################################
AL_MS_MRIP_station_points_df = pd.concat([AL_MRIP_station_points_df, MS_MRIP_station_points_df], ignore_index=True, join='outer')
AL_MS_MRIP_station_points_df = add_h3_and_neighbors_to_dataframe(AL_MS_MRIP_station_points_df, resolution)

AL_MS_MRIP_station_points_df['Site_id'] = AL_MS_MRIP_station_points_df['STATE_CODE']*10^7 +  \
                    AL_MS_MRIP_station_points_df['COUNTY_CODE']*10^4 + AL_MS_MRIP_station_points_df['SITE_EXTERNAL_ID']  ## This is Jesse's newsiteID
columns_to_retain = [
    'Site_id', 'SITE_EXTERNAL_ID', 'SITE_NAME', 'SITE_ADDRESS', 'SITE_CITY', 'SITE_ZIP',
    'COUNTY_CODE', 'COUNTY', 'STATE_CODE', 'STATE', 'lat', 'lng', 'h3_cell', 'h3_neighbors'
]

# columns_to_retain = ['Site_id', 'SITE_NAME','STATE_CODE', 'lat', 'lng']
AL_MS_MRIP_station_points_df = AL_MS_MRIP_station_points_df[columns_to_retain]
AL_MS_MRIP_station_points_df.rename(columns={'SITE_NAME':  'Site_Name', 'STATE_CODE': 'State_Code'}, inplace=True)
# Add h3 cells and neighors to the MRIP data frames
columns_to_retain = ['Site_id', 'Site_Name','State_Code', 'lat', 'lng']
AL_MS_MRIP_station_points_df = AL_MS_MRIP_station_points_df[columns_to_retain]
AL_MS_MRIP_station_points_df = add_h3_and_neighbors_to_dataframe(AL_MS_MRIP_station_points_df, resolution)

#############################################################################################################################
# TX_station_points_df

columns_to_retain = ['major_area_code', 'station_code', 'Site_name', 'lat', 'lng']
TX_station_points1_df = TX_station_points_df[columns_to_retain]
# TX_station_points1_df = add_h3_and_neighbors_to_dataframe(TX_station_points1_df, resolution)


TX_station_points1_df.rename(columns={'Site_name':  'Site_Name'}, inplace=True)
TX_station_points1_df['Site_id']=TX_station_points_df['major_area_code']*1000+ TX_station_points_df['station_code']
TX_station_points1_df['State_Code'] = 48
columns_to_retain = ['Site_id', 'Site_Name','State_Code', 'lat', 'lng']

TX_station_points1_df = TX_station_points1_df[columns_to_retain]
TX_station_points1_df = add_h3_and_neighbors_to_dataframe(TX_station_points1_df, resolution)


#############################################################################################################################
# LA_CREEL_station_points_df
columns_to_retain = ['Site_Id', 'Site_Name', 'lat', 'lng']
LA_CREEL_station_points1_df = LA_CREEL_station_points_df[columns_to_retain]
LA_CREEL_station_points1_df.rename(columns={'Site_Id':  'Site_id'}, inplace=True)
LA_CREEL_station_points1_df['State_Code'] = 22
columns_to_retain = ['Site_id', 'Site_Name','State_Code', 'lat', 'lng']
LA_CREEL_station_points1_df = LA_CREEL_station_points1_df[columns_to_retain]
LA_CREEL_station_points1_df = add_h3_and_neighbors_to_dataframe(LA_CREEL_station_points1_df, resolution)

#############################################################################################################################
for index, row in filtered_rec_indicators_df.iterrows():
    print("len(filtered_rec_indicators_df)", len(filtered_rec_indicators_df))
    assert ii == 2200000
    #############
    ii = ii +1/500
    if abs(int(ii) - ii) < 1/500:
        print(ii*500)
    #############
    # print("irow", irow)
    # intersection_results_df = []
    cuebiq_id = row['cuebiq_id']
    Trip_number = row['Trip_number']
    trip_start_epochtime = row['timestamp_start_t']
    trip_end_epochtime = row['timestamp_end_t']
        
    # Get pings before trip
    this_trip_before_df = V3_Pings_df[
        (V3_Pings_df['cuebiq_id'] == cuebiq_id) &
        (V3_Pings_df['event_timestamp'] >= (trip_start_epochtime-Eight_hours)) &
        (V3_Pings_df['event_timestamp'] <= trip_start_epochtime) ]
    this_trip_before_df=EliminateErrantPingsSpeed(this_trip_before_df, 90)

    # Get pings after trip
    this_trip_after_df = V3_Pings_df[
        (V3_Pings_df['cuebiq_id'] == cuebiq_id) &
        (V3_Pings_df['event_timestamp'] <= (trip_end_epochtime+Eight_hours)) &
        (V3_Pings_df['event_timestamp'] >= trip_end_epochtime)     ]
    this_trip_after_df=EliminateErrantPingsSpeed(this_trip_after_df, 90)

    before_and_after_df = pd.concat([this_trip_before_df, this_trip_after_df], ignore_index=True)
    before_and_after_df = before_and_after_df[(before_and_after_df['Avg_ping_speed'] <= One_mph)]
    
    # Get h3 cells of all points where the device is "stopped"
    before_and_after_h3_cells, before_and_after_df = get_h3_cells_set_and_dataframe(before_and_after_df, resolution)

    inTX = inLA = inMS = inAL = False

    AL_MRIP_station_points_intersection = list(set(before_and_after_h3_cells).intersection(AL_MRIP_station_points_h3_cells))
    if len(AL_MRIP_station_points_intersection) >0:
        inAL = True
    MS_MRIP_station_points_intersection = list(set(before_and_after_h3_cells).intersection(MS_MRIP_station_points_h3_cells))
    if len(MS_MRIP_station_points_intersection) >0:
        inMS = True
    LA_station_points_intersection = list(set(before_and_after_h3_cells).intersection(LA_CREEL_station_points_h3_cells))
    if len(LA_station_points_intersection) >0:
        inLA = True
    TX_station_points_intersection = list(set(before_and_after_h3_cells).intersection(TX_station_points_h3_cells))
    if len(TX_station_points_intersection) >0:
        inTX = True

    filtered_before_and_after_df = before_and_after_df[
        before_and_after_df['h3_cell'].isin(AL_MRIP_station_points_intersection) |
        before_and_after_df['h3_cell'].isin(LA_station_points_intersection) |
        before_and_after_df['h3_cell'].isin(TX_station_points_intersection) |
        before_and_after_df['h3_cell'].isin(MS_MRIP_station_points_intersection)]

   
    filtered_before_and_after_df['time_before_after'] = np.maximum(
        filtered_before_and_after_df['event_timestamp'] - trip_end_epochtime,
        trip_start_epochtime - filtered_before_and_after_df['event_timestamp']
    )        
    specific_h3_cell = filtered_before_and_after_df.loc[filtered_before_and_after_df['time_before_after'].idxmin(), 'h3_cell']

    if inAL or inMS:
        matching_rows = AL_MS_MRIP_station_points_df[
            (AL_MS_MRIP_station_points_df['h3_cell'] == specific_h3_cell) | 
            (AL_MS_MRIP_station_points_df['h3_neighbors'].apply(lambda x: specific_h3_cell in x))
        ]

    if inLA:
        matching_rows = LA_CREEL_station_points1_df[
            (LA_CREEL_station_points1_df['h3_cell'] == specific_h3_cell) | 
            (LA_CREEL_station_points1_df['h3_neighbors'].apply(lambda x: specific_h3_cell in x))
        ]

        
    if inTX:
        matching_rows = TX_station_points1_df[
            (TX_station_points1_df['h3_cell'] == specific_h3_cell) | 
            (TX_station_points1_df['h3_neighbors'].apply(lambda x: specific_h3_cell in x))
        ]

    
    # Select only the first row in the event that there are 2 stations very close to each other
    matching_rows = matching_rows.iloc[0]

    ######################## Now combine the necessary data for final output to a csv file
    stop_row = stops_indicators_df[
        (stops_indicators_df['cuebiq_id'] == cuebiq_id) & 
        (stops_indicators_df['Trip_number'] == Trip_number)
    ]
    if not stop_row.empty:  # Check if matching row is found
        stop_row = stop_row.iloc[0]  # Get the first (and likely only) matching row

    # Step 3: Combine columns into a single row (dictionary or list)
    combined_row = {
        'cuebiq_id': cuebiq_id,
        'Trip_number': Trip_number,  
        'trip_start_epochtime': row['timestamp_start_t'],
        'trip_end_epochtime': row['timestamp_end_t'],
        # Columns from stops_indicators_df
        'Interruption_01': stop_row['Interruption_01'],
        'nPings': stop_row['nPings'],
        'nStops': stop_row['nStops'],
        'MS': stop_row['MS'],
        'AL': stop_row['AL'],
        'FL': stop_row['FL'],
        'TX': stop_row['TX'],
        'LA': stop_row['LA'],
        'AL_MRIP_station_points_0_1': stop_row['AL_MRIP_station_points_0_1'],
        'MS_MRIP_station_points_0_1': stop_row['MS_MRIP_station_points_0_1'],
        'LA_CREEL_station_points_0_1': stop_row['LA_CREEL_station_points_0_1'],
        'TX_station_points_0_1':stop_row['TX_station_points_0_1'],
    }
   # Add the matching_row values (the other columns from matching_rows)
    for col in matching_rows.index:
        if col not in combined_row:  # Avoid duplicates
            combined_row[col] = matching_rows[col]

    combined_row_df = pd.DataFrame([combined_row])
    combined_row_df.to_csv(Stations_with_stops_filename, index=False, mode='a', 
                           header=not os.path.isfile(Stations_with_stops_filename))
print("Done")
print(ii*500," of ", len(filtered_rec_indicators_df), "remaining trips completed")


### Load and prepare the station visits based on stops analysis (TX, AL & MS)

In [ ]:
Stations_with_stops_filename = ValidationAnalysis_sites_With_Stops_filename

#####################################################################
# Load Station with Stops data used in station-nonstation analysis
Stations_with_stops_df=pd.read_csv(Stations_with_stops_filename)

Stations_with_stops_df['datetime'] = pd.to_datetime(Stations_with_stops_df['trip_start_epochtime'], unit='s')

# Create new columns
Stations_with_stops_df['date'] = Stations_with_stops_df['datetime'].dt.date
Stations_with_stops_df['year'] = Stations_with_stops_df['datetime'].dt.year
Stations_with_stops_df['month'] = Stations_with_stops_df['datetime'].dt.month
Stations_with_stops_df['day_of_week'] = Stations_with_stops_df['datetime'].dt.weekday  # 0 = Monday, 6 = Sunday
Stations_with_stops_df['year_month'] = (Stations_with_stops_df['year'])*100 + Stations_with_stops_df['month']

Stations_with_stops_df['New_Site_ID'] = 10000*Stations_with_stops_df['State_Code']+Stations_with_stops_df['Site_id']
Stations_with_stops_df['WAVE'] = ((Stations_with_stops_df['month'] - 1) // 2) + 1

Stations_with_stops_df['year_wave'] = Stations_with_stops_df['year']*100 + Stations_with_stops_df['WAVE']
Stations_with_stops_df['Year_month'] = Stations_with_stops_df['year']*100 + Stations_with_stops_df['month']

###################################################################################################
# Select list for two regions
Stations_with_stops_TX_df = Stations_with_stops_df[Stations_with_stops_df['TX_station_points_0_1']==1]
Stations_with_stops_AL_MS_df = Stations_with_stops_df[(Stations_with_stops_df['AL_MRIP_station_points_0_1']==1) |
                                                      (Stations_with_stops_df['MS_MRIP_station_points_0_1']==1) ]

###################################################################################################
# Total station trips for each region by site & month or wave
Total_mobility_visits_by_site_TX = Stations_with_stops_TX_df.groupby('Site_id').size().reset_index(name='Total_mobility_visits')
Total_mobility_visits_by_month_TX = Stations_with_stops_TX_df.groupby('year_month').size().reset_index(name='Total_mobility_visits')

Total_mobility_visits_by_site_AL_MS = Stations_with_stops_AL_MS_df.groupby('New_Site_ID').size().reset_index(name='Total_mobility_visits')
Total_mobility_visits_by_year_wave_AL_MS = Stations_with_stops_AL_MS_df.groupby('year_wave').size().reset_index(name='Total_mobility_visits')

###################################################################################################
# Calculate Comple_visits (number of rows per Site_id where Interruption_01 == 1)
complete_visits_TX = Stations_with_stops_TX_df[Stations_with_stops_TX_df['Interruption_01'] == 1] \
    .groupby('Site_id').size().reset_index(name='complete_visits')

# Calculate Comple_visits (number of rows per Site_id where Interruption_01 == 1)
complete_visits_by_month_TX = Stations_with_stops_TX_df[Stations_with_stops_TX_df['Interruption_01'] == 1] \
    .groupby('year_month').size().reset_index(name='complete_visits')

# Calculate Comple_visits (number of rows per Site_id where Interruption_01 == 1)
complete_visits_AL_MS = Stations_with_stops_AL_MS_df[Stations_with_stops_AL_MS_df['Interruption_01'] == 1] \
    .groupby('New_Site_ID').size().reset_index(name='complete_visits')

# Calculate Comple_visits (number of rows per Site_id where Interruption_01 == 1)
complete_visits_by_year_wave_AL_MS = Stations_with_stops_AL_MS_df[Stations_with_stops_AL_MS_df['Interruption_01'] == 1] \
    .groupby('year_wave').size().reset_index(name='complete_visits')

###################################################################################################
# Merge the two DataFrames on 'Site_id'
Mobility_Visits_TX_df = pd.merge(Total_mobility_visits_by_site_TX, complete_visits_TX, on='Site_id', how='left')
Mobility_Visits_TX_df['complete_visits'] = Mobility_Visits_TX_df['complete_visits'].fillna(0).astype(int)

Mobility_Visits_TX_by_month_df = pd.merge(Total_mobility_visits_by_month_TX, complete_visits_by_month_TX, on='year_month', how='left')
Mobility_Visits_TX_by_month_df['complete_visits'] = Mobility_Visits_TX_by_month_df['complete_visits'].fillna(0).astype(int)

# Merge the two DataFrames on 'Site_id'
Mobility_Visits_AL_MS_by_site_df = pd.merge(Total_mobility_visits_by_site_AL_MS, complete_visits_AL_MS, on='New_Site_ID', how='left')
Mobility_Visits_AL_MS_by_site_df['complete_visits'] = Mobility_Visits_AL_MS_by_site_df['complete_visits'].fillna(0).astype(int)

Mobility_Visits_AL_MS_by_year_wave_df = pd.merge(Total_mobility_visits_by_year_wave_AL_MS, complete_visits_by_year_wave_AL_MS, on='year_wave', how='left')
Mobility_Visits_AL_MS_by_year_wave_df['complete_visits'] = Mobility_Visits_AL_MS_by_year_wave_df['complete_visits'].fillna(0).astype(int)

# Extract unique Site_id with their lat and lng
site_coordinates_TX = Stations_with_stops_TX_df[['Site_id', 'lat', 'lng']].drop_duplicates()
site_coordinates_AL_MS = Stations_with_stops_AL_MS_df[['New_Site_ID', 'lat', 'lng']].drop_duplicates()

# Merge site coordinates with Mobility_Visits_TX_df
Mobility_Visits_TX_df = pd.merge(Mobility_Visits_TX_df, site_coordinates_TX, on='Site_id', how='left')
Mobility_Visits_AL_MS_by_site_df = pd.merge(Mobility_Visits_AL_MS_by_site_df, site_coordinates_AL_MS, on='New_Site_ID', how='left')

# Mobility_Visits_TX_by_month_df

### Load and prepare TX Roving Counts data

In [ ]:
# Texas Roving Counts data were provided by Mark Fisher (TPWD) on 5/31/2022

TX_Roving_Counts_df = pd.read_csv(TX_Roving_Counts_filename)

########################################### Clean up TX_Roving_Counts ###############################
TX_Roving_Counts_df['roving_date'] = pd.to_datetime(
    TX_Roving_Counts_df['COMPLETION_DTTM'], format='%d%b%Y:%H:%M:%S'
)
# limit to counts in 2019 and beyond
TX_Roving_Counts_df = TX_Roving_Counts_df[TX_Roving_Counts_df['roving_date'].dt.year >= 2019]
# Complete Site_id
TX_Roving_Counts_df['Site_id'] = TX_Roving_Counts_df['MAJOR_AREA_CODE']*1000 + \
                                TX_Roving_Counts_df['STATION_CODE']

TX_Roving_Counts_df = TX_Roving_Counts_df.drop(columns=['MAJOR_AREA_CODE', 'COMPLETION_DTTM', 'STATION_CODE', 'COUNT_TIME_NUM'])
TX_Roving_Counts_df = TX_Roving_Counts_df.rename(columns={'TOTAL_COUNT_NUM': 'BoatCt'})

TX_Roving_Counts_df['Site_id_count'] = TX_Roving_Counts_df.groupby('Site_id')['Site_id'].transform('count')
########################################### Aggregation by Site_id ###############################
TX_Roving_Average_by_site_df = TX_Roving_Counts_df.groupby('Site_id', as_index=False).agg(
    Avg_Roving_Ct=('BoatCt', 'mean'),
    Total_Roving_Ct=('BoatCt', 'sum'),
    Count_of_Roving_Ct=('BoatCt', 'count')
)

########################################### Aggregation by year_month ################################
TX_Roving_Counts_df['year'] = TX_Roving_Counts_df['roving_date'].dt.year
TX_Roving_Counts_df['month'] = TX_Roving_Counts_df['roving_date'].dt.month
# day_of_week ==> 0 = Monday, 6 = Sunday
TX_Roving_Counts_df['day_of_week'] = TX_Roving_Counts_df['roving_date'].dt.weekday  
TX_Roving_Counts_df['year_month'] = (TX_Roving_Counts_df['year'])*100 + TX_Roving_Counts_df['month']

TX_Roving_Average_by_month_df = TX_Roving_Counts_df.groupby('year_month', as_index=False).agg(
    Avg_BoatCt=('BoatCt', 'mean'),
    Total_BoatCt=('BoatCt', 'sum'),
    Count_of_Roving_Ct=('BoatCt', 'count')
)
TX_Roving_Average_by_month_df.rename(columns={'Avg_Roving_Ct': 'FullWeek_Avg', 
                                              'Total_Roving_Ct': 'FullWeek_Tot', 
                                              'Count_of_Roving_Ct': 'FullWeek_CountObservations',
                                             }, inplace=True)
######################################################################
# Weekend Counts
TX_Weekend_df = TX_Roving_Counts_df[
    (TX_Roving_Counts_df['day_of_week'] >= 5)]

# Calculate the average for the filtered rows grouped by 'year_month'
Weekend_avg = TX_Weekend_df.groupby('year_month', as_index=False).agg(
    Weekend_Avg_Ct=('BoatCt', 'mean'),
    Weekend_Tot_Ct=('BoatCt', 'sum'),
)

# Merge the filtered averages into the aggregated DataFrame
TX_Roving_Average_by_month_df = TX_Roving_Average_by_month_df.merge(
    Weekend_avg, on='year_month', how='left'
)

######################################################################
# Weekend for consistently active sites
site_ct_min = 44 # 90% of the sites have 44 observations or more with a maximum of 50
TX_Weekend_df = TX_Weekend_df[
    (TX_Weekend_df['Site_id_count'] >= site_ct_min)]

# Calculate the average for the filtered rows grouped by 'year_month'
Weekend_avg = TX_Weekend_df.groupby('year_month', as_index=False).agg(
    Weekend_Avg_Ct_Active=('BoatCt', 'mean'),
    Weekend_Tot_Ct_Active=('BoatCt', 'sum'),
)

# Merge the filtered averages into the aggregated DataFrame
TX_Roving_Average_by_month_df = TX_Roving_Average_by_month_df.merge(
    Weekend_avg, on='year_month', how='left'
)

######################################################################
# Weekday Counts
TX_Weekday_df = TX_Roving_Counts_df[
    (TX_Roving_Counts_df['day_of_week'] < 5)]

# Calculate the average for the filtered rows grouped by 'year_month'
TX_Weekday_avg = TX_Weekday_df.groupby('year_month', as_index=False).agg(
    TX_Weekday_avg_Ct=('BoatCt', 'mean'),
    Weekday_Tot_Ct=('BoatCt', 'sum'),
)

# Merge the filtered averages into the aggregated DataFrame
TX_Roving_Average_by_month_df = TX_Roving_Average_by_month_df.merge(
    TX_Weekday_avg, on='year_month', how='left'
)

######################################################################
# Weekday for consistently active sites
site_ct_min = 44 # 90% of the sites have 44 observations or more with a maximum of 50

TX_Weekday_df = TX_Weekday_df[
    (TX_Weekday_df['Site_id_count'] >= site_ct_min)]

# Calculate the average for the filtered rows grouped by 'year_month'
TX_Weekday_avg = TX_Weekday_df.groupby('year_month', as_index=False).agg(
    TX_Weekday_avg_Ct_Active=('BoatCt', 'mean'),
    Weekday_Tot_Ct_Active=('BoatCt', 'sum'),
)

# Merge the filtered averages into the aggregated DataFrame
TX_Roving_Average_by_month_df = TX_Roving_Average_by_month_df.merge(
    TX_Weekday_avg, on='year_month', how='left'
)



TX_Roving_Average_by_month_df.head(1)



### Load and prepare AL & MS MRIP Trips data

Create unifying site ID identifier, New_Site_ID for other MRIP data frames

In [ ]:
# Create New_Site_ID for other MRIP data frames
Stations_with_stops_AL_MS_df['New_Site_ID'] = 10000*Stations_with_stops_AL_MS_df['State_Code']+Stations_with_stops_AL_MS_df['Site_id']
MRIP_station_points_df['New_Site_ID'] = 10000*MRIP_station_points_df['STATE_CODE']+MRIP_station_points_df['SITE_EXTERNAL_ID']
# MRIP_station_points_df.columns

Create MRIP data set for calibration -- this is trips data for charter and private-rental boats in Alabama and Mississippi

In [ ]:
# MRIP data can be 
import pandas as pd
from pathlib import Path

# Base directory
base_path = Path(r"S:\Database\MRIP")

# Columns to keep
cols_to_keep = [
    "strat_id", "psu_id", "YEAR", "ST", "CNTY", "INTSITE", "MODE_FX",
    "AREA_X", "PARTY", "WAVE", "kod", "month", "TIME", "DIST",
    "ART_REEF", "wp_int"
]

# List of filenames (without .csv)
file_stems = [
    "trip_20191", "trip_20192", "trip_20193", "trip_20194", "trip_20195", "trip_20196",
    "trip_20201", "trip_20202", "trip_20203", "trip_20204", "trip_20205", "trip_20206",
    "trip_20211", "trip_20212", "trip_20213", "trip_20214", "trip_20215", "trip_20216",
    "trip_20221", "trip_20222", "trip_20223", "trip_20224", "trip_20225", "trip_20226"
]

# Initialize empty list for storing filtered DataFrames
dfs = []

# Loop over all files
for stem in file_stems:
    file_path = base_path / f"{stem}.csv"
    print(f"Loading {file_path}...")
    df = pd.read_csv(file_path, usecols=cols_to_keep)

    # Apply filters
    df = df[df["MODE_FX"].isin([5, 7])]
    df = df[df["ST"].isin([1, 28])]

    dfs.append(df)

# Concatenate all filtered DataFrames
AL_MS_MRIP_Trips_df = pd.concat(dfs, ignore_index=True)

# Optional: save combined dataset
AL_MS_MRIP_Trips_df.to_csv(MRIP_trips_filename, index=False)


Load MRIP data and create unifying site ID identifier, New_Site_ID

In [ ]:
# If the MRIP data is already processed, it can be loaded directly
AL_MS_MRIP_Trips_df = pd.read_csv(MRIP_trips_filename)
print(MRIP_trips_filename)
AL_MS_MRIP_Trips_df

In [ ]:
AL_MS_MRIP_Trips_df = pd.read_csv(MRIP_trips_filename)

# Clean up and create new variables
AL_MS_MRIP_Trips_df['New_Site_ID'] = 10000*AL_MS_MRIP_Trips_df['ST']+AL_MS_MRIP_Trips_df['INTSITE']

AL_MS_MRIP_Trips_df['year_wave'] = (AL_MS_MRIP_Trips_df['YEAR'])*100 + AL_MS_MRIP_Trips_df['WAVE']
AL_MS_MRIP_Trips_df = AL_MS_MRIP_Trips_df[AL_MS_MRIP_Trips_df['year_wave']<=202202]
                                          
AL_MS_MRIP_Trips_df = AL_MS_MRIP_Trips_df.drop(columns=['strat_id', 'psu_id'])

# Group by 'New_Site_ID' and count the unique values of 'year_wave'
Site_visits_df = AL_MS_MRIP_Trips_df.groupby('New_Site_ID')['year_wave'].nunique().reset_index()
Site_visits_df.columns = ['New_Site_ID', 'Visits_by_enumerators']

# Average values per interview -- all trips
AL_MS_MRIP_by_site_df = AL_MS_MRIP_Trips_df.groupby('New_Site_ID', as_index=False).agg(
    TripCount_All=("wp_int", "size"),  # Count the number of trips
    WeightedTrips_All=("wp_int", "sum"),  # Sum the 'wp_int' values for each group
    Interviews=("WAVE", "nunique")  # Count the number of unique 'WAVE' values
)
AL_MS_MRIP_by_site_df['TripCount_per_visit'] = AL_MS_MRIP_by_site_df['TripCount_All']/AL_MS_MRIP_by_site_df['Interviews']
AL_MS_MRIP_by_site_df['WeightedTrips_per_visit'] = AL_MS_MRIP_by_site_df['WeightedTrips_All']/AL_MS_MRIP_by_site_df['Interviews']

# Average values per interview -- offshore trips only (DIST == 2 => > 3 miles )
AL_MS_MRIP_Offshore_Trips_df=AL_MS_MRIP_Trips_df[AL_MS_MRIP_Trips_df['DIST']==2]

AL_MS_MRIP_Offshore_by_site_df = AL_MS_MRIP_Offshore_Trips_df.groupby('New_Site_ID', as_index=False).agg(
    TripCount_All=("wp_int", "size"),  # Count the number of trips
    WeightedTrips_All=("wp_int", "sum"),  # Sum the 'wp_int' values for each group
    Interviews=("WAVE", "nunique")  # Count the number of unique 'WAVE' values
)
AL_MS_MRIP_Offshore_by_site_df['TripCount_per_visit'] = AL_MS_MRIP_Offshore_by_site_df['TripCount_All']/AL_MS_MRIP_Offshore_by_site_df['Interviews']
AL_MS_MRIP_Offshore_by_site_df['WeightedTrips_per_visit'] = AL_MS_MRIP_Offshore_by_site_df['WeightedTrips_All']/AL_MS_MRIP_Offshore_by_site_df['Interviews']

AL_MS_MRIP_Offshore_by_site_df.head(1)

In [ ]:
# Distribution of roving count data
import pandas as pd
import matplotlib.pyplot as plt

# Drop duplicates to include only one value per Site_id
unique_site_counts_TX = TX_Roving_Counts_df.drop_duplicates(subset='Site_id')['Site_id_count']

# Sort the unique Site_id_count values
sorted_counts = unique_site_counts_TX.sort_values()

# Calculate the CDF
cdf = sorted_counts.rank(method='average', pct=True)

# Plot the CDF
plt.figure(figsize=(8, 6))
plt.plot(sorted_counts, cdf, marker='.', linestyle='none')

# Add horizontal grid lines every 5%
plt.yticks(ticks=[i / 20 for i in range(21)])  # 0, 0.05, 0.10, ..., 1.00
plt.grid(axis='y', linestyle='--', color='gray', alpha=0.7)

# Add labels and title
plt.title('Cumulative Distribution Function of Site_id_count')
plt.xlabel('Site_id_count')
plt.ylabel('CDF')
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Extract the data
# data = AL_MS_MRIP_Offshore_by_site_df['TripCount_per_visit']
data = AL_MS_MRIP_by_site_df['WeightedTrips_per_visit']

# Sort the data
sorted_data = np.sort(data)

# Calculate the ECDF
ecdf = np.arange(1, len(sorted_data) + 1) / len(sorted_data)

# Plot the ECDF
plt.figure(figsize=(8, 6))
plt.step(sorted_data, ecdf, where="post", color='blue', label='ECDF of TripCount_per_visit')
plt.xlabel('TripCount_per_visit')
plt.ylabel('ECDF')
plt.title('Empirical Cumulative Distribution Function (ECDF)')
plt.grid(True)
plt.legend()
plt.show()

### TX: Create Clusters for Relevant Site_id's

In [ ]:
from sklearn.cluster import DBSCAN
import numpy as np
import pandas as pd


# Load TX_station_points_df and reduce to only those rows that appear in either the roving counts or mobility data df's
TX_station_points_df['Site_id'] = (TX_station_points_df['major_area_code']*1000 + TX_station_points_df['station_code']).astype(int)
TX_relevant_site_ids = set(TX_Roving_Counts_df['Site_id']).union(Mobility_Visits_TX_df['Site_id'])
Relevant_TX_stations_df = TX_station_points_df[TX_station_points_df['Site_id'].isin(TX_relevant_site_ids)]

# Prepare the data: Extract latitude and longitude
coords = Relevant_TX_stations_df[['lat', 'lng']].values  # lat, lng columns

# Convert degrees to radians (required for haversine distance)
coords = np.radians(coords)

# Set eps for 1 km (0.0001569 radians corresponds to 1 km)
km_boundary = 1
eps_value = 0.0001569*km_boundary

# Apply DBSCAN with Haversine distance (metric='haversine' works with radians)
db = DBSCAN(eps=eps_value, min_samples=1, metric='haversine')  # Adjust eps and min_samples

# Fit DBSCAN
Relevant_TX_stations_df['cluster'] = db.fit_predict(coords)

Station_Clusters_df = Relevant_TX_stations_df[['Site_id', 'cluster', 'lat', 'lng']]

# Print the number of sites in each cluster
cluster_counts = Station_Clusters_df['cluster'].value_counts()
Station_Clusters_df['sites_by_Cluster'] = Station_Clusters_df['cluster'].map(cluster_counts)

# Print the cluster counts (excluding noise points labeled as -1)
print("There are ", len(cluster_counts), "for the ", len(Station_Clusters_df), "unique stations")
# Station_Clusters_df.head(3)

### TX: Add clusters to Roving counts and create df by cluster

In [ ]:
# Add the cluster to TX_Roving_Counts_df and Mobility_Visits_TX_df
TX_Roving_Average_by_site_df2 = TX_Roving_Average_by_site_df.merge(
    Station_Clusters_df[['Site_id', 'cluster', 'lat', 'lng']],  # Columns to merge
    on='Site_id',                                # Merge key
    how='left'                                   # Retain all rows in TX_Roving_Counts_df
)
# Columns in TX_Roving_Average_by_site_df
##              Avg_Roving_Ct	Total_Roving_Ct	Count_of_Roving_Ct

# Group by 'cluster' and apply aggregation functions
TX_Roving_Counts_by_cluster_df = TX_Roving_Average_by_site_df2.groupby('cluster', as_index=False).agg({
    'lat': 'mean',                    # Average latitude
    'lng': 'mean',                    # Average longitude
    'Avg_Roving_Ct': 'sum',         # Sum of TOTAL_COUNT_NUM
    'Count_of_Roving_Ct': 'sum',                # Count of rows in each cluster
    'Total_Roving_Ct': 'sum'                # Sum of all 
}).rename(columns={
    'Avg_Roving_Ct': 'ClusterTotal_Avg_Roving_Ct',  # Rename TOTAL_COUNT_NUM to Roving_Ct_Sum
    'Count_of_Roving_Ct': 'Num_of_Counts',             # Rename cluster (count) to Num_of_Cts
    'Total_Roving_Ct': 'All_Boats_Counted'             # Rename cluster (count) to Num_of_Cts
})
         
unique_clusters_count = TX_Roving_Counts_by_cluster_df['cluster'].nunique()
print(f"The number of unique clusters is: {unique_clusters_count}")


### TX: Add clusters to visits and create df by cluster

In [ ]:
# Mobility_Visits_TX_df.head(6)
# Add the cluster to TX_Roving_Counts_df and Mobility_Visits_TX_df
Mobility_Visits_TX_df2 = Mobility_Visits_TX_df.merge(
    Station_Clusters_df[['Site_id', 'cluster']],  # Columns to merge
    on='Site_id',                                # Merge key
    how='left'                                   # Retain all rows in TX_Roving_Counts_df
)

Mobility_Visits_by_cluster_df = Mobility_Visits_TX_df2.groupby('cluster', as_index=False).agg({
    'lat': 'mean',                    # Average latitude
    'lng': 'mean',                    # Average longitude
    'Total_mobility_visits': 'sum',            # Sum of Total_mobility_visits
    'complete_visits': 'sum',         # Sum of complete_visits
    'Site_id': 'count'                # Count of Site_id (for example, count of rows per cluster)
}).rename(columns={
    'Total_mobility_visits': 'Total_mobility_visits_by_cluster',  # Rename to descriptive names
    'complete_visits': 'complete_visits_by_cluster',
    'Site_id': 'sites_by_Cluster'               # Rename to 'sites_by_Cluster'
})

unique_clusters_count = Mobility_Visits_by_cluster_df['cluster'].nunique()
print(f"The number of unique clusters is: {unique_clusters_count}")


In [ ]:
# Get the unique 'cluster' values from both DataFrames
clusters_mobility = set(Mobility_Visits_by_cluster_df['cluster'])
clusters_roving = set(TX_Roving_Counts_by_cluster_df['cluster'])

# Find the intersection (clusters that appear in both DataFrames)
common_clusters = clusters_mobility & clusters_roving

# Find the clusters in Mobility_Visits_by_cluster_df but not in TX_Roving_Counts_by_cluster_df
mobility_only_clusters = clusters_mobility - clusters_roving

# Find the clusters in TX_Roving_Counts_by_cluster_df but not in Mobility_Visits_by_cluster_df
roving_only_clusters = clusters_roving - clusters_mobility

# Print the counts
print("with km_boundary = ", km_boundary)
print(f"Number of clusters in both Mobility_Visits_by_cluster_df and TX_Roving_Counts_by_cluster_df: {len(common_clusters)}")
print(f"Number of clusters in Mobility_Visits_by_cluster_df but not in TX_Roving_Counts_by_cluster_df: {len(mobility_only_clusters)}")
print(f"Number of clusters in TX_Roving_Counts_by_cluster_df but not in Mobility_Visits_by_cluster_df: {len(roving_only_clusters)}")


## Clusters for MRIP data in AL and MS

In [ ]:
## Add Lat & lng to site lists

# Perform the merge operation
AL_MS_MRIP_by_site_df = AL_MS_MRIP_by_site_df.drop(columns=['lat', 'lng'], errors='ignore')
AL_MS_MRIP_by_site_df = AL_MS_MRIP_by_site_df.merge(
    MRIP_station_points_df[['New_Site_ID', 'lat', 'lng']],  # Select only relevant columns
    on='New_Site_ID',  # Merge on the 'New_Site_ID' column
    how='left'  # Perform a left join to keep all rows from AL_MS_MRIP_Offshore_by_site_df
)
# MS site # 32 does not appear in any other data sets. I suspect this is an error in the underlying data
AL_MS_MRIP_by_site_df = AL_MS_MRIP_by_site_df.dropna(subset=['lat'])


# Perform the merge operation
AL_MS_MRIP_Offshore_by_site_df = AL_MS_MRIP_Offshore_by_site_df.drop(columns=['lat', 'lng'], errors='ignore')
AL_MS_MRIP_Offshore_by_site_df = AL_MS_MRIP_Offshore_by_site_df.merge(
    MRIP_station_points_df[['New_Site_ID', 'lat', 'lng']],  # Select only relevant columns
    on='New_Site_ID',  # Merge on the 'New_Site_ID' column
    how='left'  # Perform a left join to keep all rows from AL_MS_MRIP_Offshore_by_site_df
)
# MS site # 32 does not appear in any other data sets. I suspect this is an error in the underlying data
AL_MS_MRIP_Offshore_by_site_df = AL_MS_MRIP_Offshore_by_site_df.dropna(subset=['lat'])
AL_MS_MRIP_Offshore_by_site_df.head(2)


In [ ]:
from sklearn.cluster import DBSCAN
import numpy as np
import pandas as pd


# Prepare the data: Extract latitude and longitude
coords = AL_MS_MRIP_by_site_df[['lat', 'lng']].values  # lat, lng columns

# Convert degrees to radians (required for haversine distance)
coords = np.radians(coords)

# Set eps for 1 km (0.0001569 radians corresponds to 1 km)
km_boundary = 1
eps_value = 0.0001569*km_boundary

# Apply DBSCAN with Haversine distance (metric='haversine' works with radians)
db = DBSCAN(eps=eps_value, min_samples=1, metric='haversine')  # Adjust eps and min_samples

# Fit DBSCAN
AL_MS_MRIP_by_site_df['cluster'] = db.fit_predict(coords)
AL_MS_MRIP_by_site_df
AL_MS_MRIP_Clusters_df = AL_MS_MRIP_by_site_df[['New_Site_ID', 'cluster', 'lat', 'lng']]

# Print the number of sites in each cluster
cluster_counts = AL_MS_MRIP_Clusters_df['cluster'].value_counts()
AL_MS_MRIP_Clusters_df['sites_by_Cluster'] = AL_MS_MRIP_Clusters_df['cluster'].map(cluster_counts)

## Add in the average lat and lgn for each cluster
cluster_avg = AL_MS_MRIP_Clusters_df.groupby('cluster')[['lat', 'lng']].mean().reset_index()

# Rename the columns
cluster_avg.rename(columns={'lat': 'cluster_lat', 'lng': 'cluster_lng'}, inplace=True)

# Merge the averaged values back into the original DataFrame
AL_MS_MRIP_Clusters_df = AL_MS_MRIP_Clusters_df.merge(cluster_avg, on='cluster', how='left')


# Print the cluster counts (excluding noise points labeled as -1)
print("There are ", len(cluster_counts), "for the ", len(AL_MS_MRIP_Clusters_df), "unique stations")
AL_MS_MRIP_Clusters_df.head(1)

In [ ]:
# Add Clusters to the MRIP trips data
AL_MS_MRIP_Trips_df = AL_MS_MRIP_Trips_df.drop(columns=['cluster', 'lat', 'lng', 'cluster_lat', 'cluster_lng'], errors='ignore')

AL_MS_MRIP_Trips_df = AL_MS_MRIP_Trips_df.merge(
    AL_MS_MRIP_Clusters_df[['New_Site_ID', 'cluster', 'lat', 'lng', 'cluster_lat', 'cluster_lng']],  # Select only relevant columns
    on='New_Site_ID',  # Merge on the 'New_Site_ID' column
    how='left'  # Perform a left join to keep all rows from AL_MS_MRIP_Offshore_by_site_df
)

##################################################################################################################
# Group by 'New_Site_ID' and count the unique values of 'year_wave'
Cluster_visits_df = AL_MS_MRIP_Trips_df.groupby('cluster')['year_wave'].nunique().reset_index()
Cluster_visits_df.columns = ['cluster', 'Visits_by_enumerators']

##################################################################################################################
# Average values per interview -- all trips
AL_MS_MRIP_by_cluster_df = AL_MS_MRIP_Trips_df.groupby('cluster', as_index=False).agg(
    TripCount_All=("wp_int", "size"),  # Count the number of trips
    WeightedTrips_All=("wp_int", "sum"),  # Sum the 'wp_int' values for each group
    Interviews=("WAVE", "nunique")  # Count the number of unique 'WAVE' values
)
AL_MS_MRIP_by_cluster_df['TripCount_per_visit'] = AL_MS_MRIP_by_cluster_df['TripCount_All']/AL_MS_MRIP_by_cluster_df['Interviews']
AL_MS_MRIP_by_cluster_df['WeightedTrips_per_visit'] = AL_MS_MRIP_by_cluster_df['WeightedTrips_All']/AL_MS_MRIP_by_cluster_df['Interviews']

##################################################################################################################
# Average values per interview -- grouped by cluster and year_wave
AL_MS_MRIP_by_year_wave_df = AL_MS_MRIP_Trips_df.groupby(['year_wave'], as_index=False).agg(
    TripCount_All=("wp_int", "size"),  # Count the number of trips
    WeightedTrips_All=("wp_int", "sum"),  # Sum the 'wp_int' values for each group
    Interviews=("year_wave", "nunique")  # Count the number of unique 'WAVE' values
)

# Calculate per-visit averages
AL_MS_MRIP_by_year_wave_df['TripCount_per_visit'] = (
    AL_MS_MRIP_by_year_wave_df['TripCount_All'] / AL_MS_MRIP_by_year_wave_df['Interviews']
)
AL_MS_MRIP_by_year_wave_df['WeightedTrips_per_visit'] = (
    AL_MS_MRIP_by_year_wave_df['WeightedTrips_All'] / AL_MS_MRIP_by_year_wave_df['Interviews']
)

##################################################################################################################
##################### Repeat for offshore trips ###############################
# Average values per interview -- offshore trips only (DIST == 2 => > 3 miles )
AL_MS_MRIP_Offshore_Trips_df=AL_MS_MRIP_Trips_df[AL_MS_MRIP_Trips_df['DIST']==2]

AL_MS_MRIP_Offshore_by_cluster_df = AL_MS_MRIP_Offshore_Trips_df.groupby('cluster', as_index=False).agg(
    TripCount_All=("wp_int", "size"),  # Count the number of trips
    WeightedTrips_All=("wp_int", "sum"),  # Sum the 'wp_int' values for each group
    Interviews=("WAVE", "nunique")  # Count the number of unique 'WAVE' values
)
AL_MS_MRIP_Offshore_by_cluster_df['TripCount_per_visit'] = AL_MS_MRIP_Offshore_by_cluster_df['TripCount_All']/AL_MS_MRIP_Offshore_by_cluster_df['Interviews']
AL_MS_MRIP_Offshore_by_cluster_df['WeightedTrips_per_visit'] = AL_MS_MRIP_Offshore_by_cluster_df['WeightedTrips_All']/AL_MS_MRIP_Offshore_by_cluster_df['Interviews']

##################################################################################################################
# Average values per interview -- grouped by cluster and year_wave
AL_MS_MRIP_Offshore_by_year_wave_df = AL_MS_MRIP_Offshore_Trips_df.groupby(['year_wave'], as_index=False).agg(
    TripCount_All=("wp_int", "size"),  # Count the number of trips
    WeightedTrips_All=("wp_int", "sum"),  # Sum the 'wp_int' values for each group
    Interviews=("year_wave", "nunique")  # Count the number of unique 'WAVE' values
)

# Calculate per-visit averages
AL_MS_MRIP_Offshore_by_year_wave_df['TripCount_per_visit'] = (
    AL_MS_MRIP_Offshore_by_year_wave_df['TripCount_All'] / AL_MS_MRIP_Offshore_by_year_wave_df['Interviews']
)
AL_MS_MRIP_Offshore_by_year_wave_df['WeightedTrips_per_visit'] = (
    AL_MS_MRIP_Offshore_by_year_wave_df['WeightedTrips_All'] / AL_MS_MRIP_Offshore_by_year_wave_df['Interviews']
)



AL_MS_MRIP_Offshore_by_cluster_df.head(1)

# MRIP Validation
### AL & MS Add clusters to mobility data visits and create df by cluster

In [ ]:
# Merge Mobility data with cluster
if 'Mobility_Visits_AL_MS_by_site_df2' in globals():
    del Mobility_Visits_AL_MS_by_site_df2

# Merge the DataFrames while ensuring existing columns are replaced
Mobility_Visits_AL_MS_by_site_df2 = Mobility_Visits_AL_MS_by_site_df.merge(
    AL_MS_MRIP_Clusters_df[['New_Site_ID', 'cluster', 'cluster_lat', 'cluster_lng']],  
    on='New_Site_ID',  
    how='left'  
)
# print(Mobility_Visits_AL_MS_by_site_df2.columns)

Mobility_Visits_AL_MS_by_cluster_df = Mobility_Visits_AL_MS_by_site_df2.groupby('cluster', as_index=False).agg({
    'cluster_lat': 'mean',                    # Average latitude
    'cluster_lng': 'mean',                    # Average longitude
    'Total_mobility_visits': 'sum',            # Sum of Total_mobility_visits
    'complete_visits': 'sum',         # Sum of complete_visits
    'New_Site_ID': 'count'                # Count of Site_id (for example, count of rows per cluster)
}).rename(columns={
    'Total_mobility_visits': 'Total_mobility_visits_by_cluster',  # Rename to descriptive names
    'complete_visits': 'complete_mobility_visits_by_cluster',
    'New_Site_ID': 'sites_per_Cluster'               # Rename to 'sites_by_Cluster'
})

# unique_clusters_count = Mobility_Visits_by_cluster_df['cluster'].nunique()
# print(f"The number of unique clusters is: {unique_clusters_count}")
Mobility_Visits_AL_MS_by_cluster_df.head(1)

## MRIP Validation over space

In [ ]:
print("## MRIP Validation over space")
print("")

import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import spearmanr
from scipy.stats import pearsonr

### RUN OPTIONS
# # ALL TRIPS UNWEIGHTED
MRIP_df = AL_MS_MRIP_by_cluster_df
x_col = 'TripCount_per_visit' #'WeightedTrips_per_visit'  # Change this to use a different X variable
x_title = 'MRIP access point surveys'
print(x_title, len(MRIP_df))

# ALL TRIPS WEIGHTED
# MRIP_df = AL_MS_MRIP_by_cluster_df
# x_col = 'WeightedTrips_per_visit' #''  # Change this to use a different X variable
# x_title = 'MRIP weighted  access point surveys per cluster'
# print(x_title, len(MRIP_df))

# # OFFSHORE TRIPS UNWEIGHTED
# MRIP_df = AL_MS_MRIP_Offshore_by_cluster_df
# x_col = 'TripCount_per_visit' #'WeightedTrips_per_visit'  # Change this to use a different X variable
# x_title = 'Offshore trips recorded in MRIP access point surveys per cluster'
# print(x_title, len(MRIP_df))

# # OFFSHORE TRIPS WEIGHTED
# MRIP_df = AL_MS_MRIP_Offshore_by_cluster_df
# x_col = 'WeightedTrips_per_visit' #''  # Change this to use a different X variable
# x_title = 'MRIP Weighted offshore trips access point surveys per cluster'
# print(x_title, len(MRIP_df))

print("")

y_col = 'Total_mobility_visits_by_cluster'  # Change this to use a different Y variable
y_title = 'mobility data visits'  # Change this to use a different Y variable



# # MERGING WITH ALL MRIP TRIPS
# merged_df = Mobility_Visits_AL_MS_by_cluster_df.merge(
#     MRIP_df[['cluster', 'WeightedTrips_per_visit', 'TripCount_per_visit']],
#     on='cluster',
#     how='inner'  # Use 'inner' to keep only matching clusters
# )

# MERGING WITH OFFSHORE MRIP TRIPS
merged_df = Mobility_Visits_AL_MS_by_cluster_df.merge(
    MRIP_df[['cluster', 'WeightedTrips_per_visit', 'TripCount_per_visit']],
    on='cluster',
    how='inner'  # Use 'inner' to keep only matching clusters
)


# print(merged_df.columns)

import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

# Take log10 of selected columns
x = np.log10(merged_df[x_col])
y = np.log10(merged_df[y_col])
correlation = np.corrcoef(x, y)[0, 1]

# Calculate Pearson and Spearman correlations and p-values
r, p_val = pearsonr(x, y)
print(f"Pearson correlation: {r:.3f} (p = {p_val:.3g})")
spearman_corr, spearman_pval = spearmanr(x, y)
print(f"Spearman correlation: {spearman_corr:.4f} (p-value: {spearman_pval:.4g})")

# Fit the linear regression model
x_with_const = sm.add_constant(x)  # Adds an intercept term
model = sm.OLS(y, x_with_const)
results = model.fit()

# Extract regression results
coefficients = results.params
standard_errors = results.bse
trendline = results.predict(x_with_const)

slope = coefficients[x_col]
slope_se = standard_errors[x_col]


# Increase font sizes for the plot
font_size = 18
plt.rcParams.update({'font.size': font_size})

# Create the scatter plot
plt.figure(figsize=(8, 6))
plt.scatter(x, y, color='blue', alpha=0.7, label='Data Points')


# Plot the trend line
plt.plot(x, trendline, color='red', linestyle='--', label='Trend Line')

# Add labels and title dynamically
plt.xlabel(f'Log (10) of {x_title}', fontsize=font_size)
plt.ylabel(f'Log (10) of {y_title}', fontsize=font_size)

###################
y_min, y_max = plt.ylim()  # Get the y-axis limits
x_min, x_max = plt.xlim()  # Get the x-axis limits

plt.text(
    x_max,                       # x coordinate at the rightmost point
    y_min + 0.07*(y_max - y_min),  # keep your desired vertical offset
    f"Correlation coefficient: {correlation:.2f}",
    color='black',
    fontsize=font_size,
    ha='right'                   # anchor text to its right edge
)
##################

plt.legend()
# Show the grid for better readability
plt.grid(alpha=0.3)

plt.show()


###################### Conduct a Chi-Squared contingency test ############

# Define columns dynamically

# Create a contingency table using selected columns
contingency_table = pd.DataFrame({
    'Visits_data': merged_df[x_col],
    'Counts_data': merged_df[y_col]
})

# Perform the chi-squared test
from scipy.stats import chi2_contingency
chi2_stat, p_value, dof, expected = chi2_contingency(contingency_table)

# Print the coefficients of the fitted line
print("Coefficients of fitted line", coefficients)

# Print the results
print("\nResults of the Chi-Squared contingency test")
print("Chi-squared Statistic:", chi2_stat)
print("P-value:", p_value)
print("Degrees of Freedom:", dof)

# Interpret the results
alpha = 0.01  # Significance level
if p_value < alpha:
    print(f"At the {alpha*100:.0f}% level, reject the null hypothesis. The data are likely not generated by the same process.")
else:
    print(f"At the {alpha*100:.0f}% level, fail to reject the null hypothesis. The data may come from the same process.")


## Validation graph -- MRIP variation over time

In [ ]:
print("## Validation graph -- MRIP variation over time")
print("")
print("")

import pandas as pd
import matplotlib.pyplot as plt

### RUN OPTIONS
GraphName = "ALL TRIPS UNWEIGHTED"
MRIP_df = AL_MS_MRIP_by_year_wave_df
x_col = 'TripCount_per_visit' #'WeightedTrips_per_visit'  # Change this to use a different X variable
x_title = 'MRIP access point surveys'

# GraphName = "ALL TRIPS WEIGHTED"
# MRIP_df = AL_MS_MRIP_by_year_wave_df
# x_col = 'WeightedTrips_per_visit' #''  # Change this to use a different X variable
# x_title = 'weighted MRIP access point surveys'

# GraphName = "OFFSHORE TRIPS UNWEIGHTED"
# MRIP_df = AL_MS_MRIP_Offshore_by_year_wave_df
# x_col = 'TripCount_per_visit' #'WeightedTrips_per_visit'  # Change this to use a different X variable
# x_title = 'offshore MRIP access point surveys'

# GraphName = "OFFSHORE TRIPS WEIGHTED"
# MRIP_df = AL_MS_MRIP_Offshore_by_year_wave_df
# x_col = 'WeightedTrips_per_visit' #''  # Change this to use a different X variable
# x_title = 'offshore weighted MRIP access point surveys'

merged_df = Mobility_Visits_AL_MS_by_year_wave_df.merge(
    MRIP_df[['year_wave', 'WeightedTrips_per_visit', 'TripCount_per_visit']],
    on='year_wave',
    how='inner'  # Use 'inner' to keep only matching clusters
)
# print(merged_df.columns)

import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

# Choose columns dynamically
y_col = 'Total_mobility_visits'  # Change this to use a different Y variable
y_title = 'mobility data visits'  # Change this to use a different Y variable

# Take log10 of selected columns
x = np.log10(merged_df[x_col])
# print(merged_df[y_col].min())
y = np.log10(merged_df[y_col])
correlation = np.corrcoef(x, y)[0, 1]

# Calculate Pearson and Spearman correlations and p-values
r, p_val = pearsonr(x, y)
print(f"Pearson correlation: {r:.3f} (p = {p_val:.3g})")
spearman_corr, spearman_pval = spearmanr(x, y)
print(f"Spearman correlation: {spearman_corr:.4f} (p-value: {spearman_pval:.4g})")


# Fit the linear regression model
x_with_const = sm.add_constant(x)  # Adds an intercept term
model = sm.OLS(y, x_with_const)
results = model.fit()

# Extract regression results
coefficients = results.params
standard_errors = results.bse
trendline = results.predict(x_with_const)

slope = coefficients[x_col]
slope_se = standard_errors[x_col]

# Increase font sizes for the plot
font_size = 18
plt.rcParams.update({'font.size': font_size})

# Create the scatter plot
plt.figure(figsize=(8, 6))
plt.scatter(x, y, color='blue', alpha=0.7, label='Data Points')

# Plot the trend line
plt.plot(x, trendline, color='red', linestyle='--', label='Trend Line')

# print (GraphName)
# Add labels and title dynamically
plt.xlabel(f'Log (10) of {x_title}', fontsize=font_size)
plt.ylabel(f'Log (10) of {y_title}', fontsize=font_size)
###################
# y_min, y_max = plt.ylim()  # Get the y-axis limits
# x_min, x_max = plt.xlim()  # Get the x-axis limits

# plt.text((x_min + x_max) / 1.75, y_max * 0.25, 
#          f"Coefficient (slope): {slope:.2f}\nStandard Error: {slope_se:.2f}", 
#          color='black', fontsize=12, ha='center')
y_min, y_max = plt.ylim()  # Get the y-axis limits
x_min, x_max = plt.xlim()  # Get the x-axis limits

# plt.text(x_min+(x_max-x_min)*.8, y_min+(y_max-y_min)*.1, 
#          f"Correlation coefficient: {correlation:.2f}", 
#          color='black', fontsize=12, ha='center')

plt.text(
    x_max,                       # x coordinate at the rightmost point
    y_min + 0.07*(y_max - y_min),  # keep your desired vertical offset
    f"Correlation coefficient: {correlation:.2f}",
    color='black',
    fontsize=font_size,
    ha='right'                   # anchor text to its right edge
)
##################
plt.grid(alpha=0.3)

plt.legend()
plt.show()


###################### Conduct a Chi-Squared contingency test ############

# Define columns dynamically

# Create a contingency table using selected columns
contingency_table = pd.DataFrame({
    'Visits_data': merged_df[x_col],
    'Counts_data': merged_df[y_col]
})

# Perform the chi-squared test
from scipy.stats import chi2_contingency
chi2_stat, p_value, dof, expected = chi2_contingency(contingency_table)

# Print the coefficients of the fitted line
print("Coefficients of fitted line", coefficients)

# Print the results
print("\nResults of the Chi-Squared contingency test")
print("Chi-squared Statistic:", chi2_stat)
print("P-value:", p_value)
print("Degrees of Freedom:", dof)

# Interpret the results
alpha = 0.01  # Significance level
if p_value < alpha:
    print(f"At the {alpha*100:.0f}% level, reject the null hypothesis. The data are likely not generated by the same process.")
else:
    print(f"At the {alpha*100:.0f}% level, fail to reject the null hypothesis. The data may come from the same process.")


# TXRC Analysis

In [ ]:
### TX ANALYSIS
# Get the unique 'cluster' values from both DataFrames
clusters_mobility = set(Mobility_Visits_by_cluster_df['cluster'])
clusters_roving = set(TX_Roving_Counts_by_cluster_df['cluster'])

# Find the intersection (clusters that appear in both DataFrames)
common_clusters = clusters_mobility & clusters_roving

# Find the clusters in Mobility_Visits_by_cluster_df but not in TX_Roving_Counts_by_cluster_df
mobility_only_clusters = clusters_mobility - clusters_roving

# Find the clusters in TX_Roving_Counts_by_cluster_df but not in Mobility_Visits_by_cluster_df
roving_only_clusters = clusters_roving - clusters_mobility

# Print the counts
print("with km_boundary = ", km_boundary)
print(f"Number of clusters in both Mobility_Visits_by_cluster_df and TX_Roving_Counts_by_cluster_df: {len(common_clusters)}")
print(f"Number of clusters in Mobility_Visits_by_cluster_df but not in TX_Roving_Counts_by_cluster_df: {len(mobility_only_clusters)}")
print(f"Number of clusters in TX_Roving_Counts_by_cluster_df but not in Mobility_Visits_by_cluster_df: {len(roving_only_clusters)}")


### TX: Create single df with one row per cluster, including only clusters with non-zero counts and visits

In [ ]:
# Perform the merge
Visits_and_Counts_by_cluster_df = pd.merge(
    Mobility_Visits_by_cluster_df, 
    TX_Roving_Counts_by_cluster_df, 
    on='cluster', 
    how='outer',  # 'outer' merge to retain all rows from both DataFrames
    suffixes=('', '_drop')  # Use a suffix to identify duplicate columns from the second DataFrame
)

# Replace NaNs in 'lat' and 'lng' columns with values from '_drop' columns (if they exist)
Visits_and_Counts_by_cluster_df['lat'] = Visits_and_Counts_by_cluster_df['lat'].combine_first(
    Visits_and_Counts_by_cluster_df['lat_drop']
)
Visits_and_Counts_by_cluster_df['lng'] = Visits_and_Counts_by_cluster_df['lng'].combine_first(
    Visits_and_Counts_by_cluster_df['lng_drop']
)

# Drop duplicate columns (from TX_Roving_Counts_by_cluster_df)
Visits_and_Counts_by_cluster_df = Visits_and_Counts_by_cluster_df.loc[:, ~Visits_and_Counts_by_cluster_df.columns.str.endswith('_drop')]

# Replace missing values (NaNs) with 0
Visits_and_Counts_by_cluster_df.fillna(0, inplace=True)


Visits_and_Counts_non_zero_by_cluster_df = Visits_and_Counts_by_cluster_df[Visits_and_Counts_by_cluster_df['Total_mobility_visits_by_cluster']>0]
Visits_and_Counts_non_zero_by_cluster_df = Visits_and_Counts_non_zero_by_cluster_df[Visits_and_Counts_non_zero_by_cluster_df['ClusterTotal_Avg_Roving_Ct']>0]

# Display the resulting DataFrame
print("Of a total of ", len(Visits_and_Counts_by_cluster_df), "clusters, ", len(Visits_and_Counts_non_zero_by_cluster_df), "have positive visits and counts")


#### Show clusters where counts are found but no visits

In [ ]:
import folium

# Create a base map centered at the average location in the dataset
center_lat = Visits_and_Counts_by_cluster_df['lat'].mean()
center_lng = Visits_and_Counts_by_cluster_df['lng'].mean()
map_clusters = folium.Map(location=[center_lat, center_lng], zoom_start=8)

# Add points to the map
for _, row in Visits_and_Counts_by_cluster_df.iterrows():
    # Determine marker color based on Total_mobility_visits_by_cluster
    color = 'blue'
    color = 'red' if row['Total_mobility_visits_by_cluster'] == 0 else color
    color = 'green' if row['ClusterTotal_Avg_Roving_Ct'] == 0 else color
    
    # Add a marker
    folium.CircleMarker(
        location=(row['lat'], row['lng']),
        radius=3,  # Adjust point size if needed
        color=color,
        fill=True,
        fill_opacity=0.8,
        tooltip = f"Cluster: {int(row['cluster'])} \
                    \nTotal Visits: {int(row['Total_mobility_visits_by_cluster'])} \
                    \nAvg Ct: {int(row['ClusterTotal_Avg_Roving_Ct'])}"
    ).add_to(map_clusters)

# Display the map
map_clusters


## TXRC Validation graph over space

In [ ]:
### CORRECTED X AND Y AXES

print("## TXRC Validation graph over space")
print("")

import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

# Choose columns dynamically
y_col = 'Total_mobility_visits_by_cluster'  # Change this to use a different Y variable
y_title = 'mobility data visits'  # Change this to use a different Y variable
x_col = 'ClusterTotal_Avg_Roving_Ct' #'WeightedTrips_per_visit'  # Change this to use a different X variable
x_title = 'Texas roving count per site'

## copy      x_col =                                             'ClusterTotal_Avg_Roving_Ct' #'WeightedTrips_per_visit'  # Change this to use a different X variable
## one above x = np.log(Visits_and_Counts_non_zero_by_cluster_df['ClusterTotal_Avg_Roving_Ct'])

# Extract x and y values
# x = np.log10(Visits_and_Counts_non_zero_by_cluster_df['Total_mobility_visits_by_cluster'])
# y = np.log10(Visits_and_Counts_non_zero_by_cluster_df['ClusterTotal_Avg_Roving_Ct'])

x = np.log10(Visits_and_Counts_non_zero_by_cluster_df[x_col])
y = np.log10(Visits_and_Counts_non_zero_by_cluster_df[y_col])
correlation = np.corrcoef(x, y)[0, 1]

# Calculate Pearson and Spearman correlations and p-values
r, p_val = pearsonr(x, y)
print(f"Pearson correlation: {r:.3f} (p = {p_val:.3g})")
spearman_corr, spearman_pval = spearmanr(x, y)
print(f"Spearman correlation: {spearman_corr:.4f} (p-value: {spearman_pval:.4g})")


# x = sm.add_constant(x)  # Adds a constant (intercept) term to the independent variable
# Fit the linear regression model
x_with_const = sm.add_constant(x)
model = sm.OLS(y, x_with_const)  # OLS = Ordinary Least Squares
results = model.fit()  # Fit the model

coefficients = results.params  # [intercept, slope]
# print(coefficients)
standard_errors = results.bse  # [standard error of intercept, standard error of slope]
trendline = results.predict(x_with_const)

slope = coefficients[x_col]  # Use the appropriate name
slope_se = standard_errors[x_col]  # Use the appropriate name

# Increase font sizes for the plot
font_size = 18
plt.rcParams.update({'font.size': font_size})

# Create the scatter plot
plt.figure(figsize=(8, 6))
plt.scatter(x, y, color='blue', alpha=0.7, label='Data Points')

# print("Coefficient (slope): {coefficients[1]:.2f}\nStandard Error: {standard_errors[1]:.2f}")

# Plot the trend line
plt.plot(x, trendline, color='red', linestyle='--', label='Trend Line')

# Add labels, title, and legend
plt.xlabel(f'Log (10) of {x_title}', fontsize=font_size)
plt.ylabel(f'Log (10) of {y_title}', fontsize=font_size)
# plt.xlabel('Log (10) of Texas roving counts', fontsize=font_size)
# plt.ylabel('Log (10) of mobility data visits', fontsize=font_size)
# plt.title('Predicted trips vs. roving counts per cluster', fontsize=14)

# Annotate the plot with the coefficient and standard error
# plt.text(0.95, 0.05, f"Coefficient (slope): {slope:.2f}\nStandard Error: {slope_se:.2f}", 
#          color='black', fontsize=12, ha='right', va='bottom', transform=plt.gca().transAxes)
y_min, y_max = plt.ylim()  # Get the y-axis limits
x_min, x_max = plt.xlim()  # Get the x-axis limits

# plt.text(x_min+(x_max-x_min)*.8, y_min+(y_max-y_min)*.1, 
#          f"Correlation coefficient: {correlation:.2f}", 
#          color='black', fontsize=12, ha='center')

plt.text(
    x_max,                       # x coordinate at the rightmost point
    y_min + 0.07*(y_max - y_min),  # keep your desired vertical offset
    f"Correlation coefficient: {correlation:.2f}",
    color='black',
    fontsize=font_size,
    ha='right'                   # anchor text to its right edge
)

plt.legend()

# Show the grid for better readability
plt.grid(alpha=0.3)

# Display the plot
plt.show()

###################### Conduct a Chi-Squared contingency test ############
# Create a contingency table
contingency_table = pd.DataFrame({
    'Visits_data': Visits_and_Counts_non_zero_by_cluster_df[x_col],
    'Counts_data': Visits_and_Counts_non_zero_by_cluster_df[y_col]
})

# Perform the chi-squared test
from scipy.stats import chi2_contingency
chi2_stat, p_value, dof, expected = chi2_contingency(contingency_table)

# Print the coefficients of the fitted line
print("Coefficients of fitted line", coefficients)

# Print the results
print("\nResults of the Chi-Squared contingency test")
print("Chi-squared Statistic:", chi2_stat)
print("P-value:", p_value)
print("Degrees of Freedom:", dof)
# Interpret the results
alpha = 0.01  # Significance level
if p_value < alpha:
    print(f"At the {alpha*100:.0f}% level, reject the null hypothesis. The data are likely not generated by the same process.")
else:
    print(f"At the {alpha*100:.0f}% level, fail to reject the null hypothesis. The data may be generated by the same process.")


### TX: Create single df with one row per year-month


In [ ]:
# Perform the merge
Visits_and_Counts_by_year_month_df = pd.merge(
    Mobility_Visits_TX_by_month_df, 
    TX_Roving_Average_by_month_df, 
    on='year_month', 
    how='outer',  # 'outer' merge to retain all rows from both DataFrames
    suffixes=('', '_drop')  # Use a suffix to identify duplicate columns from the second DataFrame
)

# Drop duplicate columns (from TX_Roving_Counts_by_cluster_df)
Visits_and_Counts_by_year_month_df = Visits_and_Counts_by_year_month_df.loc[:, ~Visits_and_Counts_by_year_month_df.columns.str.endswith('_drop')]

# Replace missing values (NaNs) with 0
Visits_and_Counts_by_year_month_df.fillna(0, inplace=True)


# print("Of a total of ", len(Visits_and_Counts_by_year_month_df), "year_month combinations, ")
Visits_and_Counts_by_year_month_df.head(1)

## TXRC Validation graphs over time

In [ ]:

print("## TXRC Validation graphs over time")
print("")

import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt


# BOAT COUNT OPTIONS
Ct_variable = 'Avg_BoatCt'
Ct_Title = "Texas roving counts"

# Ct_variable = 'Total_BoatCt'  
# Ct_Title = "sum of all roving counts"

# Ct_variable = 'Weekend_Avg_Ct'  
# Ct_Title = "average weekend roving count at all sites"

# Ct_variable = 'Weekend_Tot_Ct'  
# Ct_Title = "sum of all weekend roving counts"

# Ct_variable = 'Weekend_Avg_Ct_Active'
# Ct_Title = "average weekend roving roving count at active sites"

# Ct_variable = 'Weekend_Tot_Ct_Active' 
# Ct_Title = "sum of all weekend roving counts at active sites"

# Ct_variable = 'TX_Weekday_avg_Ct'
# Ct_Title = "average weekday roving count at all sites"

# Ct_variable = 'Weekday_Tot_Ct' 
# Ct_Title = "sum of all weekday roving counts at all sites"

# Ct_variable = 'TX_Weekday_avg_Ct_Active'
# Ct_Title = "average weekday roving count at active sites"

# Ct_variable = 'Weekday_Tot_Ct_Active' 
# Ct_Title = "sum of all weekday roving counts at active sites"

y_col = 'Total_mobility_visits'  # Change this to use a different Y variable
y_title = 'mobility data visits'  # Change this to use a different Y variable
x_col = 'Avg_BoatCt' #'WeightedTrips_per_visit'  # Change this to use a different X variable
x_title = 'Texas roving counts'

# Create data for regression
# Log specification
Visits_and_Counts_by_year_month_df1=Visits_and_Counts_by_year_month_df[
    (Visits_and_Counts_by_year_month_df[y_col]>0) &
    (Visits_and_Counts_by_year_month_df[x_col]>0)]

x = np.log10(Visits_and_Counts_by_year_month_df1[x_col])
y = np.log10( Visits_and_Counts_by_year_month_df1[y_col])
correlation = np.corrcoef(x, y)[0, 1]

# Calculate Pearson and Spearman correlations and p-values
r, p_val = pearsonr(x, y)
print(f"Pearson correlation: {r:.3f} (p = {p_val:.3g})")
spearman_corr, spearman_pval = spearmanr(x, y)
print(f"Spearman correlation: {spearman_corr:.4f} (p-value: {spearman_pval:.4g})")

visits_title='Log (10) of mobility data visits'
RovingCt_title = 'Log (10) of ' + Ct_Title

# # Non-transformed specification
# x = (Visits_and_Counts_by_year_month_df['Total_mobility_visits'])
# y = ( Visits_and_Counts_by_year_month_df['Total_BoatCt'])
# visits_title='Mobiltiy Data Visits'
# RovingCt_title ='Roving Ct'

# x = sm.add_constant(x)  # Adds a constant (intercept) term to the independent variable
# Fit the linear regression model
x_with_const = sm.add_constant(x)
model = sm.OLS(y, x_with_const)  # OLS = Ordinary Least Squares
results = model.fit()  # Fit the model

coefficients = results.params  # [intercept, slope]
# print(coefficients)
standard_errors = results.bse  # [standard error of intercept, standard error of slope]
trendline = results.predict(x_with_const)

slope = coefficients[x_col]  # Use the appropriate name
slope_se = standard_errors[x_col]  # Use the appropriate name

# Increase font sizes for the plot
font_size = 18
plt.rcParams.update({'font.size': font_size})

# Create the scatter plot
plt.figure(figsize=(8, 6))
plt.scatter(x, y, color='blue', alpha=0.7, label='Data Points')

# print("Coefficient (slope): {coefficients[1]:.2f}\nStandard Error: {standard_errors[1]:.2f}")

# Plot the trend line
plt.plot(x, trendline, color='red', linestyle='--', label='Trend Line')

# Add labels, title, and legend
plt.xlabel(RovingCt_title, fontsize=font_size)
plt.ylabel(visits_title, fontsize=font_size)
# plt.title('Predicted trips vs. roving counts per year-month', fontsize=14)
# Annotate the plot with the coefficient and standard error
# plt.text(0.95, 0.05, f"Coefficient (slope): {slope:.2f}\nStandard Error: {slope_se:.2f}", 
#          color='black', fontsize=12, ha='right', va='bottom', transform=plt.gca().transAxes)
y_min, y_max = plt.ylim()  # Get the y-axis limits
x_min, x_max = plt.xlim()  # Get the x-axis limits

plt.text(
    x_max,                       # x coordinate at the rightmost point
    y_min + 0.07*(y_max - y_min),  # keep your desired vertical offset
    f"Correlation coefficient: {correlation:.2f}",
    color='black',
    fontsize=font_size,
    ha='right'                   # anchor text to its right edge
)
plt.legend()

# Show the grid for better readability
plt.grid(alpha=0.3)

# Display the plot
plt.show()
###################### Conduct a Chi-Squared contingency test ############
# Create a contingency table
contingency_table = pd.DataFrame({
    'Visits_Data': Visits_and_Counts_by_year_month_df1['Total_mobility_visits'],
    'TXRC_Data': Visits_and_Counts_by_year_month_df1[Ct_variable]
})

# Perform the chi-squared test
chi2_stat, p_value, dof, expected = chi2_contingency(contingency_table)

# Print the coefficients of the fitted line
print("Coefficients of fitted line", coefficients)

# Print the results
print("\nResults of the Chi-Squared contingency test")
print("Chi-squared Statistic:", chi2_stat)
print("P-value:", p_value)
print("Degrees of Freedom:", dof)
# Interpret the results
alpha = 0.05  # Significance level
if p_value < alpha:
    print(f"At the {alpha*100:.2f}% level, reject the null hypothesis. The data are likely not generated by the same process.")
else:
    print(f"At the {alpha*100:.2f}% level, fail to reject the null hypothesis. The data may be generated by the same process.")
